In [ ]:
from sklearn.utils import resample
# from imblearn.over_sampling import SMOTE, SVMSMOTE
from scipy.sparse import csr_matrix

import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer, Normalizer, MaxAbsScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, train_test_split
from sklearn.decomposition import PCA, NMF, SparsePCA, KernelPCA
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, SelectPercentile, VarianceThreshold

from sklearn.svm import SVC
from fairlearn.metrics import demographic_parity_difference
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

In [ ]:
# !pip install scikit-learn
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [ ]:
classifier_S = SVC(kernel='rbf', C=1.0, gamma='scale', verbose=False)
classifier_R = RandomForestClassifier(verbose=False)
classifier_D = DecisionTreeClassifier()
classifier_G = GradientBoostingClassifier(verbose=False)
classifier_K = KNeighborsClassifier()
classifier_GNB = GaussianNB()
classifier_GB = GradientBoostingClassifier(verbose=False)
classifier_NN = MLPClassifier()
classifier_IDA = QuadraticDiscriminantAnalysis()
classifier_QDA = QuadraticDiscriminantAnalysis()
classifier_ADB = AdaBoostClassifier()
classifier_GP = GaussianProcessClassifier()
classifier_XGBC = XGBClassifier(verbosity=0)
classifier_LGBM = LGBMClassifier()
classifier_BC = BaggingClassifier(verbose=False)
classifier_CB = CatBoostClassifier(iterations=100, verbose=False)

classifiers = [
     ('SVC', classifier_S),
     ('RF', classifier_R),
     ('DT', classifier_D),
       ('GB', classifier_G),
     ('KNN', classifier_K),
       ('GNB', classifier_GNB),
     ('GBC', classifier_GB),
     ('NN', classifier_NN),
     ('IDA', classifier_IDA),
     ('QDA', classifier_QDA),  
      ('ADB', classifier_ADB),
#      ('GP', classifier_GP),
     ('XGB', classifier_XGBC),
     ('LGB', classifier_LGBM),
     ('BC', classifier_BC),
     ('CB', classifier_CB),     
]


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

In [ ]:

def Fairness_Metrics_Computation(y1, y2, y, attribute):
    # Convert inputs to pandas Series for easier handling
    y1 = pd.Series(y1)
    y2 = pd.Series(y2)
    y = pd.Series(y)
    attribute = pd.Series(attribute)

    # Identify privileged and unprivileged groups
    privileged = (attribute == 'White') | (attribute == 4)
    unprivileged = ~privileged

    # Counts of privileged and unprivileged groups
    count_privileged = np.sum(privileged)
    count_unprivileged = np.sum(unprivileged)

    # SPD Calculation
    SFC_SPD_P = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                        for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_SPD_NP = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                         for y1_i, y2_i, attr in zip(y1, y2, attribute) if attr != 'White' and attr != 4])

    SFR_SPD_p = SFC_SPD_P / count_privileged if count_privileged > 0 else 0
    SFR_SPD_np = SFC_SPD_NP / count_unprivileged if count_unprivileged > 0 else 0
    SPD = SFR_SPD_np - SFR_SPD_p

    # EOD Calculation
    SFC_EOD_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_EOD_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                         for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'White' and attr != 4])

    P_Y1 = np.sum((y == 1) & privileged)
    NP_Y1 = np.sum((y == 1) & unprivileged)

    SFR_EOD_p = SFC_EOD_P / P_Y1 if P_Y1 > 0 else 0
    SFR_EOD_np = SFC_EOD_NP / NP_Y1 if NP_Y1 > 0 else 0
    EOD = SFR_EOD_np - SFR_EOD_p

    # AOD Calculation
    SFC_TP_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_TP_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'White' and attr != 4])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'White' and attr != 4])

    P_Y0 = np.sum((y == 0) & privileged)
    NP_Y0 = np.sum((y == 0) & unprivileged)

    SFR_AOD_p = (SFC_TP_P / P_Y1 if P_Y1 > 0 else 0) + (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0)
    SFR_AOD_np = (SFC_TP_NP / NP_Y1 if NP_Y1 > 0 else 0) + (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0)
    AOD = (SFR_AOD_np - SFR_AOD_p) / 2

    # ERD Calculation
    SFC_FN_P = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_FN_NP = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'White' and attr != 4])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr == 'White' or attr == 4])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr in zip(y, y1, y2, attribute) if attr != 'White' and attr != 4])

    SFR_ERR_p = (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0) + (SFC_FN_P / P_Y1 if P_Y1 > 0 else 0)
    SFR_ERR_np = (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0) + (SFC_FN_NP / NP_Y1 if NP_Y1 > 0 else 0)
    ERD = SFR_ERR_np - SFR_ERR_p

    return SPD, EOD, AOD, ERD

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Load and preprocess the dataset
train_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.data'
test_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.test'

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
                'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']
na_values = ['?']

# Read train and test datasets
train = pd.read_csv(train_path, header=None, names=column_names, skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=None, names=column_names, skipinitialspace=True, na_values=na_values)

# Map target variable to binary
train['income-per-year'] = train['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
test['income-per-year'] = test['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

# Combine data for consistent preprocessing
combined_data = pd.concat([train, test])
combined_data.dropna(subset=['income-per-year'], inplace=True)

X_combined = combined_data.drop('income-per-year', axis=1)
y_combined = combined_data['income-per-year']

# Identify categorical and numerical columns
categorical_cols = X_combined.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_combined.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Convert categorical columns to string for OneHotEncoder compatibility
X_combined[categorical_cols] = X_combined[categorical_cols].astype(str)

# Split data into stratified and non-stratified sets
X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(
    X_combined, y_combined, test_size=0.3, random_state=42, stratify=y_combined)
X_train_no_strat, X_test_no_strat, y_train_no_strat, y_test_no_strat = train_test_split(
    X_combined, y_combined, test_size=0.3, random_state=42)

# Custom Feature Engineering Transformer
class EnhancedFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.feature_names_in_ = None

    def fit(self, X, y=None):
        if hasattr(X, 'columns'):
            self.feature_names_in_ = X.columns
        return self

    def transform(self, X):
        if hasattr(X, 'toarray'):
            X = pd.DataFrame(X.toarray(), columns=self.feature_names_in_)
        
        X_transformed = X.copy()
        
        # Log transformation, binary indicator, and interaction term
        if 'age' in X_transformed.columns:
            X_transformed['age_log'] = np.log(X_transformed['age'].replace(0, np.nan).fillna(1))
        if 'hours-per-week' in X_transformed.columns:
            X_transformed['high_hours'] = (X_transformed['hours-per-week'] > 40).astype(int)
        if 'capital-gain' in X_transformed.columns and 'capital-loss' in X_transformed.columns:
            X_transformed['capital_interaction'] = X_transformed['capital-gain'] * X_transformed['capital-loss']
        
        X_transformed.fillna(0, inplace=True)
        return X_transformed

# Define preprocessors
pca_ss_preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=5))
        ]), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

ss_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

pca_preprocessor = ColumnTransformer(
    transformers=[
        ('num', PCA(n_components=5), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Define the pipelines
pipeline_1 = Pipeline(steps=[
    ('preprocessor', pca_ss_preprocessor),
    ('feature_engineering', EnhancedFeatureEngineer()),
    ('classifier', DecisionTreeClassifier(max_depth=5, min_samples_split=10, criterion='gini'))
])

pipeline_2 = Pipeline(steps=[
    ('preprocessor', ss_preprocessor),
    ('feature_engineering', EnhancedFeatureEngineer()),
    ('classifier', DecisionTreeClassifier(max_depth=6, min_samples_split=15, criterion='entropy'))
])

pipeline_3 = Pipeline(steps=[
    ('preprocessor', pca_preprocessor),
    ('feature_engineering', EnhancedFeatureEngineer()),
    ('classifier', DecisionTreeClassifier(max_depth=4, min_samples_split=8, min_samples_leaf=5))
])

pipeline_4 = Pipeline(steps=[
    ('preprocessor', pca_ss_preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, min_samples_split=10, min_samples_leaf=5, max_features='sqrt', class_weight='balanced'))
])

# Train and evaluate the pipelines, storing results in separate variables
X_train, X_test, y_train, y_test = X_train_strat, X_test_strat, y_train_strat, y_test_strat
pipeline_1.fit(X_train, y_train)
y_pred_1 = pipeline_1.predict(X_test)
A1 = accuracy_score(y_test, y_pred_1)

pipeline_2.fit(X_train, y_train)
y_pred_2 = pipeline_2.predict(X_test)
A2 = accuracy_score(y_test, y_pred_2)

pipeline_3.fit(X_train, y_train)
y_pred_3 = pipeline_3.predict(X_test)
A3 = accuracy_score(y_test, y_pred_3)

# For Pipeline 4, use non-stratified split
X_train, X_test, y_train, y_test = X_train_no_strat, X_test_no_strat, y_train_no_strat, y_test_no_strat
pipeline_4.fit(X_train, y_train)
y_pred_4 = pipeline_4.predict(X_test)
A4 = accuracy_score(y_test, y_pred_4)

# Print results
print(f"Pipeline 1 Accuracy: {A1}")
print(f"Pipeline 2 Accuracy: {A2}")
print(f"Pipeline 3 Accuracy: {A3}")
print(f"Pipeline 4 Accuracy: {A4}")
A1

In [ ]:
A1 = accuracy_score(y_test, y_pred_1)
A2 = accuracy_score(y_test, y_pred_2)
A3 = accuracy_score(y_test, y_pred_3)
A4 = accuracy_score(y_test, y_pred_4)

f1 = f1_score(y_test, y_pred_1)
f2 = f1_score(y_test, y_pred_2)
f3 = f1_score(y_test, y_pred_3)
f4 = f1_score(y_test, y_pred_4)

In [ ]:
from sklearn.dummy import DummyClassifier

pipeline_baseline = DummyClassifier(strategy='most_frequent')
pipeline_baseline.fit(X_train, y_train)
y_pred_baseline = pipeline_baseline.predict(X_test)
mispred_baseline = (y_test != y_pred_baseline).sum()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    
    # Define the treatment variable
    treatment = X_imputed[treatment_column].apply(lambda x: 1 if x == 4 or x == 'White' else 0)

    # Drop the treatment column to create the covariate matrix
    X_covariates = X_imputed.drop(columns=[treatment_column])

    # One-hot encode the covariate matrix
    X_encoded = pd.get_dummies(X_covariates, drop_first=True)

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0]  # 'White' or 4 are considered treated
    control_indices = np.where((treatment == 0))[0]  # All others are control

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices


def flip_race(attribute):
    return ['Non-White' if x == '4' else 'White' for x in attribute]

In [ ]:
from sklearn.dummy import DummyClassifier

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=X['age'])

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

def calculate_biases(mispred_p1, mispred_p2, mispred_p3,mispred_p4, mispred_baseline):
    """
    Calculate individual biases and total bias.

    Parameters:
    mispred_p1 (int): Mispredictions for P1 (LE+SS+Classifier)
    mispred_p2 (int): Mispredictions for P2 (SS+Classifier)
    mispred_p3 (int): Mispredictions for P3 (LE+Classifier)
    mispred_baseline (int): Mispredictions for the baseline model

    Returns:
    dict: A dictionary containing the individual biases and total bias
    """
    # Calculating individual biases
    BD = mispred_baseline
    BLE = mispred_p1 - mispred_p2
    LE_SS_Interaction = mispred_p1 - (mispred_p2 + mispred_p3 - mispred_baseline)

    BSS = mispred_p1 - mispred_p3
    SS_PC_Interaction = mispred_p1 - (mispred_p2 + mispred_p4 - mispred_baseline)
    
    BPC = mispred_p1 - mispred_p4
    PC_Classifier_Interaction = mispred_p1 - (mispred_p3 + mispred_p4 - mispred_baseline)

    # Calculate BC to ensure TB equals mispred_p1
    BC = mispred_p1 - (BD + BLE + BSS + BPC + LE_SS_Interaction + SS_PC_Interaction + PC_Classifier_Interaction)

    # Calculate the total bias
    TB = BD + BLE + BSS + BPC + BC + LE_SS_Interaction + SS_PC_Interaction + PC_Classifier_Interaction

    # Returning the results as a dictionary
    return {
        'BD': BD,

        'BSt-1': BLE,
        'St1-St2_Interaction': LE_SS_Interaction,
        'BSt-2': BSS,
        'St2-St3_Interaction': SS_PC_Interaction,
        'BSt-3': BPC,
              
        'St3_Classifier_Interaction': PC_Classifier_Interaction,
        'BC': BC,
        'TB': TB
    }

import matplotlib.pyplot as plt

def plot_biases(biases, mispred_p1):
    """
    Plot biases and observed mispredictions.
    """
    labels = [
        'Data (BD)', 
        
        'Preprocessing Stage 1', 
        'Stage 1-2 Interaction',
        'Preprocessing Stage 2', 
         
       
        'Stage 2-3 Interaction',
        'Preprocessing Stage 3',
        'Stage 3-Classifier Interaction',
        'Classifier (BC)', 
        'Total Bias (TB)', 
        'Observed Bias (mispred_p1)'
    ]
    
    values = [
        biases['BD'], 
         
        biases['BSt-1'],
        biases['St1-St2_Interaction'],
        biases['BSt-2'], 
        
        
        biases['BSt-3'],
        biases['St2-St3_Interaction'], 
        biases['St3_Classifier_Interaction'], 
        biases['BC'],
        biases['TB'], 
        mispred_p1
    ]

    colors = [
        'blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray'
    ]

    plt.figure(figsize=(12, 8))
    bars = plt.barh(labels, values, color=colors)
    plt.xlabel('Bias / Mispredictions')
    plt.title('Bias Analysis and Observed Mispredictions')

    # Adding annotations
    for bar in bars:
        width = bar.get_width()
        label_x_pos = width if width < 0 else width
        plt.text(
            label_x_pos, bar.get_y() + bar.get_height() / 2,
            f'{width:.2f}', ha='center', va='center',
            bbox=dict(facecolor='white', alpha=0.7)
        )

    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlim(min(values) - 10, max(values) + 10)  # Adding some padding to the x-axis
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.show()

# Example usage with hypothetical values



# Assuming X_train, y_train, X_test, y_test, y_pred_1, y_pred_2, y_pred_3 are defined
mispred_p1 = sum(y_test != y_pred_1)
mispred_p2 = sum(y_test != y_pred_2)
mispred_p3 = sum(y_test != y_pred_3)
mispred_p4 = sum(y_test != y_pred_4)



# biases = calculate_biases_multiple(mispred_p1, mispred_p2, mispred_p3, mispred_p4, mispred_baseline)

# Calculate biases
biases = calculate_biases(mispred_p1, mispred_p2, mispred_p3,mispred_p4, mispred_baseline)
plot_biases(biases, mispred_p1)


In [ ]:
volatility_LE = np.zeros(len(y_test))
volatility_SS = np.zeros(len(y_test))
volatility_3 = np.zeros(len(y_test))
volatility_classifier = np.zeros(len(y_test))

# Volatility score calculation for PS1 (Preprocessing Stage 1)
for i in range(len(y_test)):
    if y_pred_1n[i] == y_test.iloc[i] and y_pred_2n[i] == y_test.iloc[i]:
        volatility_LE[i] = 0
    elif y_pred_1n[i] == y_test.iloc[i] and y_pred_2n[i] != y_test.iloc[i]:
        volatility_LE[i] = 1
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_2n[i] == y_test.iloc[i]:
        volatility_LE[i] = 2
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_2n[i] != y_test.iloc[i] and y_pred_1n[i] == y_pred_2n[i]:
        volatility_LE[i] = 3
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_2n[i] != y_test.iloc[i] and y_pred_1n[i] != y_pred_2n[i]:
        volatility_LE[i] = 4

# Volatility score calculation for PS2 (Preprocessing Stage 2)
for i in range(len(y_test)):
    if y_pred_1n[i] == y_test.iloc[i] and y_pred_3n[i] == y_test.iloc[i]:
        volatility_SS[i] = 0
    elif y_pred_1n[i] == y_test.iloc[i] and y_pred_3n[i] != y_test.iloc[i]:
        volatility_SS[i] = 1
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_3n[i] == y_test.iloc[i]:
        volatility_SS[i] = 2
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_3n[i] != y_test.iloc[i] and y_pred_1n[i] == y_pred_3n[i]:
        volatility_SS[i] = 3
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_3n[i] != y_test.iloc[i] and y_pred_1n[i] != y_pred_3n[i]:
        volatility_SS[i] = 4

for i in range(len(y_test)):
    if y_pred_1n[i] == y_test.iloc[i] and y_pred_4n[i] == y_test.iloc[i]:
        volatility_3[i] = 0
    elif y_pred_1n[i] == y_test.iloc[i] and y_pred_4n[i] != y_test.iloc[i]:
        volatility_3[i] = 1
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_4n[i] == y_test.iloc[i]:
        volatility_3[i] = 2
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_4n[i] != y_test.iloc[i] and y_pred_1n[i] == y_pred_4n[i]:
        volatility_3[i] = 3
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_4n[i] != y_test.iloc[i] and y_pred_1n[i] != y_pred_4n[i]:
        volatility_3[i] = 4        
        
# Volatility score calculation for classifier
for i in range(len(y_test)):
    if y_pred_1n[i] == y_test.iloc[i] and y_pred_baseline[i] == y_test.iloc[i]:
        volatility_classifier[i] = 0
    elif y_pred_1n[i] == y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i]:
        volatility_classifier[i] = 1
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_baseline[i] == y_test.iloc[i]:
        volatility_classifier[i] = 2
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i] and y_pred_1n[i] == y_pred_baseline[i]:
        volatility_classifier[i] = 3
    elif y_pred_1n[i] != y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i] and y_pred_1n[i] != y_pred_baseline[i]:
        volatility_classifier[i] = 4

# Summarize volatility scores
volatility_summary = {
    'Condition': ['Vol_PS1=0', 'Vol_PS1=1', 'Vol_PS1=2', 'Vol_PS1=3', 'Vol_PS1=4',
                  'Vol_PS2=0', 'Vol_PS2=1', 'Vol_PS2=2', 'Vol_PS2=3', 'Vol_PS2=4',
                  'Vol_PS3=0', 'Vol_PS3=1', 'Vol_PS3=2', 'Vol_PS3=3', 'Vol_PS3=4',
                  'Vol_Cl=0', 'Vol_Cl=1', 'Vol_Cl=2', 'Vol_Cl=3', 'Vol_Cl=4'],
    'Count': [np.sum(volatility_LE == 0), np.sum(volatility_LE == 1), np.sum(volatility_LE == 2), np.sum(volatility_LE == 3), np.sum(volatility_LE == 4),
              np.sum(volatility_SS == 0), np.sum(volatility_SS == 1), np.sum(volatility_SS == 2), np.sum(volatility_SS == 3), np.sum(volatility_SS == 4),
              np.sum(volatility_3 == 0), np.sum(volatility_3 == 1), np.sum(volatility_3 == 2), np.sum(volatility_3 == 3), np.sum(volatility_3 == 4),

              np.sum(volatility_classifier == 0), np.sum(volatility_classifier == 1), np.sum(volatility_classifier == 2), np.sum(volatility_classifier == 3), np.sum(volatility_classifier == 4)]
}

df_volatility = pd.DataFrame(volatility_summary)

plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Condition', y='Count', data=df_volatility, palette='viridis')

plt.title('Volatility Score Analysis - After Mitigation')
plt.xlabel('Condition')
plt.ylabel('Count')
plt.xticks(rotation=75)

for i in ax.containers:
    ax.bar_label(i, fmt='%.0f', label_type='edge')

plt.show()

In [ ]:
77+574


In [ ]:
volatility_LE = np.zeros(len(y_test))
volatility_SS = np.zeros(len(y_test))
volatility_3 = np.zeros(len(y_test))
volatility_classifier = np.zeros(len(y_test))

# Volatility score calculation for PS1 (Preprocessing Stage 1)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test.iloc[i] and y_pred_2[i] == y_test.iloc[i]:
        volatility_LE[i] = 0
    elif y_pred_1[i] == y_test.iloc[i] and y_pred_2[i] != y_test.iloc[i]:
        volatility_LE[i] = 1
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_2[i] == y_test.iloc[i]:
        volatility_LE[i] = 2
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_2[i] != y_test.iloc[i] and y_pred_1[i] == y_pred_2[i]:
        volatility_LE[i] = 3
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_2[i] != y_test.iloc[i] and y_pred_1[i] != y_pred_2[i]:
        volatility_LE[i] = 4

# Volatility score calculation for PS2 (Preprocessing Stage 2)
for i in range(len(y_test)):
    if y_pred_1[i] == y_test.iloc[i] and y_pred_3[i] == y_test.iloc[i]:
        volatility_SS[i] = 0
    elif y_pred_1[i] == y_test.iloc[i] and y_pred_3[i] != y_test.iloc[i]:
        volatility_SS[i] = 1
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_3[i] == y_test.iloc[i]:
        volatility_SS[i] = 2
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_3[i] != y_test.iloc[i] and y_pred_1[i] == y_pred_3[i]:
        volatility_SS[i] = 3
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_3[i] != y_test.iloc[i] and y_pred_1[i] != y_pred_3[i]:
        volatility_SS[i] = 4

for i in range(len(y_test)):
    if y_pred_1[i] == y_test.iloc[i] and y_pred_4[i] == y_test.iloc[i]:
        volatility_3[i] = 0
    elif y_pred_1[i] == y_test.iloc[i] and y_pred_4[i] != y_test.iloc[i]:
        volatility_3[i] = 1
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_4[i] == y_test.iloc[i]:
        volatility_3[i] = 2
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_4[i] != y_test.iloc[i] and y_pred_1[i] == y_pred_4[i]:
        volatility_3[i] = 3
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_4[i] != y_test.iloc[i] and y_pred_1[i] != y_pred_4[i]:
        volatility_3[i] = 4        
        
# Volatility score calculation for classifier
for i in range(len(y_test)):
    if y_pred_1[i] == y_test.iloc[i] and y_pred_baseline[i] == y_test.iloc[i]:
        volatility_classifier[i] = 0
    elif y_pred_1[i] == y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i]:
        volatility_classifier[i] = 1
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_baseline[i] == y_test.iloc[i]:
        volatility_classifier[i] = 2
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i] and y_pred_1[i] == y_pred_baseline[i]:
        volatility_classifier[i] = 3
    elif y_pred_1[i] != y_test.iloc[i] and y_pred_baseline[i] != y_test.iloc[i] and y_pred_1[i] != y_pred_baseline[i]:
        volatility_classifier[i] = 4

# Summarize volatility scores
volatility_summary = {
    'Condition': ['Vol_PS1=0', 'Vol_PS1=1', 'Vol_PS1=2', 'Vol_PS1=3', 'Vol_PS1=4',
                  'Vol_PS2=0', 'Vol_PS2=1', 'Vol_PS2=2', 'Vol_PS2=3', 'Vol_PS2=4',
                  'Vol_PS3=0', 'Vol_PS3=1', 'Vol_PS3=2', 'Vol_PS3=3', 'Vol_PS3=4',
                  'Vol_Cl=0', 'Vol_Cl=1', 'Vol_Cl=2', 'Vol_Cl=3', 'Vol_Cl=4'],
    'Count': [np.sum(volatility_LE == 0), np.sum(volatility_LE == 1), np.sum(volatility_LE == 2), np.sum(volatility_LE == 3), np.sum(volatility_LE == 4),
              np.sum(volatility_SS == 0), np.sum(volatility_SS == 1), np.sum(volatility_SS == 2), np.sum(volatility_SS == 3), np.sum(volatility_SS == 4),
              np.sum(volatility_3 == 0), np.sum(volatility_3 == 1), np.sum(volatility_3 == 2), np.sum(volatility_3 == 3), np.sum(volatility_3 == 4),

              np.sum(volatility_classifier == 0), np.sum(volatility_classifier == 1), np.sum(volatility_classifier == 2), np.sum(volatility_classifier == 3), np.sum(volatility_classifier == 4)]
}

df_volatility = pd.DataFrame(volatility_summary)

plt.figure(figsize=(14, 8))
ax = sns.barplot(x='Condition', y='Count', data=df_volatility, palette='viridis')

plt.title('Volatility Score Analysis')
plt.xlabel('Condition')
plt.ylabel('Count')
plt.xticks(rotation=75)

for i in ax.containers:
    ax.bar_label(i, fmt='%.0f', label_type='edge')

plt.show()

In [ ]:
from typing import Callable, Dict, List, Tuple, Any, Optional
import numpy as np
from sklearn.metrics import accuracy_score

# Optional: SMOTE if you plan to oversample (safe fallback if not installed)
try:
    from imblearn.over_sampling import SMOTE  # noqa
except Exception:
    SMOTE = None  # oversampling step will be skipped if None


# ---------- Mitigation framework (no hard-coding of stages) ----------
MitigationFn = Callable[
    [Any, np.ndarray, np.ndarray, Dict[str, Any]],
    Tuple[Any, np.ndarray, np.ndarray, Dict[str, Any]]
]
# Signature: fn(pipeline, X_train, y_train, context) -> (pipeline, X_train, y_train, fit_params_delta)

def run_mitigations_and_predict(
    pipeline: Any,
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test:  np.ndarray,
    y_test:  Optional[np.ndarray] = None,
    mitigations: Optional[List[Dict[str, Any]]] = None,
    base_fit_params: Optional[Dict[str, Any]] = None,
    context: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Fits pipeline, applies mitigation callables, then refits and re-predicts.
    Returns pre/post metrics & predictions. No random post-hoc tweaks.
    """
    base_fit_params = dict(base_fit_params or {})
    context = dict(context or {})
    mitigations = mitigations or []

    # 1) Baseline fit & predict (pre-mitigation)
    pipeline.fit(X_train, y_train, **base_fit_params)
    y_pred_pre = pipeline.predict(X_test)

    metrics_pre = {}
    if y_test is not None:
        metrics_pre["accuracy"] = float(accuracy_score(y_test, y_pred_pre))

    # 2) Apply mitigations (may update pipeline/data/fit_params)
    applied = []
    X_train_m, y_train_m = X_train, y_train
    fit_params = dict(base_fit_params)

    for step in mitigations:
        name = step.get("name", "<unnamed>")
        fn: MitigationFn = step["fn"]
        params = step.get("params", {})
        ctx = dict(context); ctx.update(params)

        pipeline, X_train_m, y_train_m, delta = fn(pipeline, X_train_m, y_train_m, ctx)
        if delta:
            fit_params.update(delta)
        applied.append({"name": name, "fit_params_delta_keys": list((delta or {}).keys())})

    # 3) Refit post-mitigation & predict
    pipeline.fit(X_train_m, y_train_m, **fit_params)
    y_pred_post = pipeline.predict(X_test)

    metrics_post = {}
    if y_test is not None:
        metrics_post["accuracy"] = float(accuracy_score(y_test, y_pred_post))

    return {
        "y_pred_pre": y_pred_pre,
        "y_pred_post": y_pred_post,
        "metrics_pre": metrics_pre,
        "metrics_post": metrics_post,
        "applied_mitigations": applied,
        "pipeline_final": pipeline
    }


# ---------- Example mitigation primitives (compose as you like) ----------
def mitigation_reweighing(pipeline, X_train, y_train, context):
    """
    Adds per-sample weights into fit params (no data mutation).
    Provide either:
      - context["sample_weights"] (np.ndarray aligned with y_train), or
      - context["weight_fn"](X_train, y_train, context) -> np.ndarray
    You can change which estimator receives weights via context["weight_param"]
      e.g., "classifier__sample_weight" or "clf__sample_weight".
    """
    if "sample_weights" in context:
        w = np.asarray(context["sample_weights"])
    elif callable(context.get("weight_fn", None)):
        w = np.asarray(context["weight_fn"](X_train, y_train, context))
    else:
        raise ValueError("mitigation_reweighing needs 'sample_weights' or 'weight_fn' in context.")

    target_param = context.get("weight_param", "classifier__sample_weight")
    return pipeline, X_train, y_train, {target_param: w}


def mitigation_oversample(pipeline, X_train, y_train, context):
    """
    Uses a provided oversampler (e.g., SMOTE) from context["oversampler"].
    """
    oversampler = context.get("oversampler", None)
    if oversampler is None:
        # If SMOTE not available or not provided, leave data unchanged
        return pipeline, X_train, y_train, {}
    X_res, y_res = oversampler.fit_resample(X_train, y_train)
    return pipeline, X_res, y_res, {}


def mitigation_encoder_tweak(pipeline, X_train, y_train, context):
    """
    Placeholder to swap/modify an encoder inside your pipeline if needed.
    Example:
      pipeline.set_params(preprocess__encoder=YourEncoder(**context.get("encoder_params", {})))
    """
    return pipeline, X_train, y_train, {}


# ---------- A simple, sensible reweighting function (group × label inverse prevalence) ----------
def inverse_prevalence_weights(X, y, ctx):
    """
    Compute weights ~ 1 / P(y, s) using ctx["s_train"] (sensitive attribute, aligned with y).
    """
    s = np.asarray(ctx.get("s_train", None))
    if s is None:
        raise ValueError("inverse_prevalence_weights requires 's_train' in context.")
    y = np.asarray(y)
    pairs, counts = np.unique(np.column_stack([y, s]), axis=0, return_counts=True)
    freq = {tuple(k): v for k, v in zip(map(tuple, pairs), counts)}
    total = len(y)
    w = np.empty_like(y, dtype=float)
    for i, (yy, ss) in enumerate(zip(y, s)):
        p = freq[(yy, ss)] / total
        w[i] = 1.0 / max(p, 1e-12)
    # Normalize to mean 1.0 (optional)
    w /= w.mean()
    return w


# =========================
# PRODUCE THE NEW PREDICTIONS
# =========================
# Assumptions: you already have:
#   pipeline_1, pipeline_2, pipeline_3  (sklearn Pipelines/estimators)
#   X_train, y_train, X_test, y_test     (arrays/Series)
#   s_train                               (sensitive attribute aligned with y_train)
#
# If you don't want oversampling, just remove the mitigation_oversample step below.

mitigations_cfg = [
    {"name": "Reweighing", "fn": mitigation_reweighing,
     "params": {
         "weight_param": "classifier__sample_weight",   # adjust to your pipeline
         "weight_fn": inverse_prevalence_weights
     }},
    {"name": "Oversample", "fn": mitigation_oversample,
     "params": {"oversampler": SMOTE(random_state=42) if SMOTE is not None else None}},
    {"name": "Encoder tweak", "fn": mitigation_encoder_tweak, "params": {}},
]

common_context = {"s_train": s_train, "seed": 42}

res_1 = run_mitigations_and_predict(
    pipeline=pipeline_1,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_1n = res_1["y_pred_post"]

res_2 = run_mitigations_and_predict(
    pipeline=pipeline_2,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_2n = res_2["y_pred_post"]

res_3 = run_mitigations_and_predict(
    pipeline=pipeline_3,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_3n = res_3["y_pred_post"]

res_4 = run_mitigations_and_predict(
    pipeline=pipeline_4,
    X_train=X_train, y_train=y_train,
    X_test=X_test,  y_test=y_test,
    mitigations=mitigations_cfg,
    base_fit_params={},
    context=common_context
)
y_pred_4n = res_4["y_pred_post"]

# (Optional) Inspect honest pre/post metrics:
# print(res_1["metrics_pre"], res_1["metrics_post"])
# print(res_2["metrics_pre"], res_2["metrics_post"])
# print(res_3["metrics_pre"], res_3["metrics_post"])


In [ ]:
from __future__ import annotations
from typing import Dict, List, Tuple, Any, Optional, Sequence
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.metrics import accuracy_score


# =========================
# Helpers for stage outputs
# =========================
def _truncate_pipeline(pipeline, stage_name: str):
    """
    Return a shallow-cloned pipeline that runs up to and including `stage_name`.
    Special names:
      - "__raw__": no transform, just return raw X
      - "__end__": full pipeline
    """
    if stage_name == "__raw__":
        return None
    if stage_name == "__end__":
        return clone(pipeline)
    steps = []
    for name, est in pipeline.steps:
        steps.append((name, est))
        if name == stage_name:
            break
    from sklearn.pipeline import Pipeline
    return Pipeline(steps=steps)

def _fit_transform_at_stage(pipeline, X: pd.DataFrame, stage_name: str) -> pd.DataFrame:
    if stage_name == "__raw__":
        return _ensure_df(X)
    tp = _truncate_pipeline(pipeline, stage_name)
    if tp is None:
        return _ensure_df(X)
    try:
        tp.set_output(transform="pandas")
    except Exception:
        pass
    Xt = tp.fit_transform(X)
    return _ensure_df(Xt)

def _ensure_df(X) -> pd.DataFrame:
    if isinstance(X, pd.DataFrame):
        return X
    if hasattr(X, "toarray"):
        X = X.toarray()
    if isinstance(X, np.ndarray):
        return pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])
    return pd.DataFrame(X)

def _expand_attributes_in_transformed(X_after: pd.DataFrame, attributes: Sequence[str]) -> List[str]:
    """Map requested raw attributes to transformed columns (handles one-hot by substring match)."""
    cols = []
    for a in attributes:
        a_low = str(a).lower()
        for c in X_after.columns:
            if a_low in str(c).lower():
                cols.append(c)
    if not cols:
        cols = [c for c in X_after.columns if c in attributes]
    # dedupe
    seen, out = set(), []
    for c in cols:
        if c not in seen:
            out.append(c); seen.add(c)
    return out


# ===========================================
# 1) Correlations before vs after any stage
# ===========================================
def compare_attribute_correlations(
    pipeline,
    X: pd.DataFrame,
    attributes: Sequence[str],
    stage_name: str,
    method: str = "pearson",
    figsize: Tuple[int, int] = (10, 4)
) -> Dict[str, pd.DataFrame]:
    """
    Compute & display correlation matrices for selected `attributes`
    BEFORE vs AFTER a given preprocessing `stage_name`.
    stage_name: "__raw__" | any pipeline step name | "__end__"
    """
    X_before = _ensure_df(X)
    X_after  = _fit_transform_at_stage(pipeline, X, stage_name)

    before_cols = [c for c in X_before.columns if c in attributes]
    after_cols  = _expand_attributes_in_transformed(X_after, attributes)

    corr_before = _ensure_df(X_before[before_cols]).corr(method=method) if before_cols else pd.DataFrame()
    corr_after  = _ensure_df(X_after[after_cols]).corr(method=method)   if after_cols  else pd.DataFrame()

    if not corr_before.empty:
        plt.figure(figsize=figsize)
        plt.imshow(corr_before, interpolation="nearest")
        plt.xticks(range(len(corr_before.columns)), corr_before.columns, rotation=90)
        plt.yticks(range(len(corr_before.index)), corr_before.index)
        plt.title(f"Correlation BEFORE stage '{stage_name}' ({method})")
        plt.colorbar()
        plt.tight_layout()
        plt.show()

    if not corr_after.empty:
        plt.figure(figsize=figsize)
        plt.imshow(corr_after, interpolation="nearest")
        plt.xticks(range(len(corr_after.columns)), corr_after.columns, rotation=90)
        plt.yticks(range(len(corr_after.index)), corr_after.index)
        plt.title(f"Correlation AFTER stage '{stage_name}' ({method})")
        plt.colorbar()
        plt.tight_layout()
        plt.show()

    return {"before": corr_before, "after": corr_after}


# ==================================================
# 2) Feature distribution before vs after any stage
# ==================================================
def compare_feature_distribution(
    pipeline,
    X: pd.DataFrame,
    feature: str,
    stage_name: str,
    bins: int = 20,
    top_k_categories: int = 25,
    figsize: Tuple[int, int] = (8, 4)
) -> Dict[str, pd.Series]:
    """
    Display the distribution of a single `feature` BEFORE vs AFTER `stage_name`.
    Numeric => histogram; Categorical => top-k bar counts.
    If transformed into multiple one-hot columns, aggregate those columns into a categorical count vector.
    """
    X_before = _ensure_df(X).copy()
    X_after  = _fit_transform_at_stage(pipeline, X, stage_name)

    # BEFORE
    if feature in X_before.columns:
        series_before = X_before[feature]
    else:
        matches = [c for c in X_before.columns if c.lower() == feature.lower()]
        series_before = X_before[matches[0]] if matches else pd.Series(dtype=float, name=feature)

    # AFTER (map to columns)
    after_cols = _expand_attributes_in_transformed(X_after, [feature])

    if len(after_cols) == 0:
        series_after = pd.Series(dtype=float, name=feature)
    elif len(after_cols) == 1 and after_cols[0] in X_after.columns:
        series_after = X_after[after_cols[0]]
    else:
        sub = X_after[after_cols]
        cat_names = [c.replace(f"{feature}_", "") for c in sub.columns]
        counts = sub.apply(pd.Series.sum, axis=0)
        counts.index = cat_names
        series_after = counts

    # Plot BEFORE
    if pd.api.types.is_numeric_dtype(series_before):
        plt.figure(figsize=figsize)
        plt.hist(series_before.dropna(), bins=bins)
        plt.title(f"Distribution BEFORE stage '{stage_name}' — {feature}")
        plt.xlabel(feature); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_before = series_before
    else:
        vc = series_before.astype("object").fillna("<NA>").value_counts().sort_values(ascending=False)
        vc = vc.head(top_k_categories)
        plt.figure(figsize=figsize)
        plt.bar(vc.index.astype(str), vc.values)
        plt.title(f"Distribution BEFORE stage '{stage_name}' — {feature}")
        plt.xticks(rotation=90); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_before = vc

    # Plot AFTER
    if isinstance(series_after, pd.Series) and pd.api.types.is_numeric_dtype(series_after) and series_after.shape[0] == X_after.shape[0]:
        plt.figure(figsize=figsize)
        plt.hist(series_after.dropna(), bins=bins)
        plt.title(f"Distribution AFTER stage '{stage_name}' — {feature}")
        plt.xlabel(feature); plt.ylabel("count")
        plt.tight_layout(); plt.show()
        dist_after = series_after
    else:
        if isinstance(series_after, pd.Series) and series_after.index.size > 0:
            counts = series_after.sort_values(ascending=False).head(top_k_categories)
            plt.figure(figsize=figsize)
            plt.bar(counts.index.astype(str), counts.values)
            plt.title(f"Distribution AFTER stage '{stage_name}' — {feature} (aggregated)")
            plt.xticks(rotation=90); plt.ylabel("count")
            plt.tight_layout(); plt.show()
            dist_after = counts
        else:
            dist_after = pd.Series(dtype=float, name=feature)

    return {"before": dist_before, "after": dist_after}


# ==========================================================
# 3) Threshold guard that calls YOUR mitigation function
#    - Uses your Fairness_Metrics_Computation(...)
#    - Calls your run_mitigations_and_predict(...) if violated
# ==========================================================
def _check_thresholds(metrics: Dict[str, float], thresholds: Dict[str, Tuple]) -> bool:
    """
    Return True if ANY threshold is violated.
    thresholds[k] = (op, val) or ("between", lo, hi)
    """
    def violated_one(metric, rule):
        op = rule[0]
        vals = rule[1:]
        m = metrics.get(metric, None)
        if m is None:
            return False
        if op == ">=":   return not (m >= vals[0])
        if op == "<=":   return not (m <= vals[0])
        if op == ">":    return not (m >  vals[0])
        if op == "<":    return not (m <  vals[0])
        if op == "between":
            lo, hi = vals
            return not (lo <= m <= hi)
        raise ValueError(f"Unknown threshold op: {op}")
    return any(violated_one(k, v) for k, v in thresholds.items())

def guard_and_mitigate_if_needed(
    pipeline,
    X_train: pd.DataFrame,
    y_train: np.ndarray,
    X_test: pd.DataFrame,
    y_test: np.ndarray,
    sensitive_attr_test: np.ndarray,
    thresholds: Dict[str, Tuple],
    # ex:
    # thresholds = {
    #   "accuracy": (">=", 0.85),
    #   "disparate_impact_ratio": (">=", 0.8),
    #   "demographic_parity_diff": ("between", -0.1, 0.1)
    # }
    mitigations_cfg: List[Dict[str, Any]],
    context: Dict[str, Any],
    base_fit_params: Optional[Dict[str, Any]] = None,
    fairness_compute_type: str = "global",
    y2_scores: Optional[np.ndarray] = None,
    positive_label=1
) -> Dict[str, Any]:
    """
    1) Fit the baseline pipeline and compute metrics
       - Accuracy via sklearn
       - Fairness via your Fairness_Metrics_Computation(y1, y2, y, attribute, compute_type)
         (y1 = y_pred, y2 = y2_scores or None, y = y_test, attribute = sensitive_attr_test)
    2) If ANY threshold is violated, call your run_mitigations_and_predict(...)
    3) Return both pre/post metrics and final predictions
    """
    base_fit_params = dict(base_fit_params or {})
    model = clone(pipeline)
    model.fit(X_train, y_train, **base_fit_params)
    y_pred = model.predict(X_test)

    # --- metrics (accuracy + fairness) ---
    metrics = {"accuracy": float(accuracy_score(y_test, y_pred))}
    fairness_dict = Fairness_Metrics_Computation(
        y1=y_pred, y2=y2_scores, y=y_test, attribute=sensitive_attr_test, compute_type=fairness_compute_type
    )
    # Merge fairness metrics into metrics dict
    metrics.update({k: float(v) for k, v in fairness_dict.items()})

    violated = _check_thresholds(metrics, thresholds)

    result = {
        "violated": violated,
        "metrics_pre": metrics,
        "y_pred_pre": y_pred,
        "pipeline_pre": model
    }

    if not violated:
        result.update({
            "used_post_mitigation": False,
            "y_pred_final": y_pred,
            "metrics_final": metrics,
            "pipeline_final": model
        })
        return result

    # --- run your mitigation routine (already defined in your codebase) ---
    mit_res = run_mitigations_and_predict(
        pipeline=model,
        X_train=X_train, y_train=y_train,
        X_test=X_test,  y_test=y_test,
        mitigations=mitigations_cfg,
        base_fit_params=base_fit_params,
        context=context
    )

    # recompute fairness on post-mitigation predictions for apples-to-apples
    y_pred_post = mit_res["y_pred_post"]
    metrics_post = {"accuracy": float(accuracy_score(y_test, y_pred_post))}
    fairness_post = Fairness_Metrics_Computation(
        y1=y_pred_post, y2=y2_scores, y=y_test, attribute=sensitive_attr_test, compute_type=fairness_compute_type
    )
    metrics_post.update({k: float(v) for k, v in fairness_post.items()})

    result.update({
        "used_post_mitigation": True,
        "mitigation_detail": mit_res.get("applied_mitigations", []),
        "y_pred_post": y_pred_post,
        "metrics_post": metrics_post,
        "y_pred_final": y_pred_post,
        "metrics_final": metrics_post,
        "pipeline_final": mit_res["pipeline_final"],
    })
    return result


In [ ]:
compare_attribute_correlations(
    pipeline=pipeline_1,
    X=X_train,
    attributes=["sex"],
    stage_name="preprocess"    # or "__raw__", "__end__", or any step name in your pipeline
)


compare_feature_distribution(
    pipeline=pipeline_2,
    X=X_train,
    feature="education",
    stage_name="preprocess"
)

thresholds = {
    "accuracy": (">=", 0.85),
    "disparate_impact_ratio": (">=", 0.8),
    "demographic_parity_diff": ("between", -0.1, 0.1),
}

res1 = guard_and_mitigate_if_needed(
    pipeline=pipeline_1,
    X_train=X_train, y_train=y_train,
    X_test=X_test,   y_test=y_test,
    sensitive_attr_test=s_test,
    thresholds=thresholds,
    mitigations_cfg=mitigations_cfg,  # the same list you pass to run_mitigations_and_predict
    context=common_context,           # e.g., {"s_train": s_train, ...}
    fairness_compute_type="global",   # or "local"
    y2_scores=None                    # provide scores if your fairness function needs them
)
y_pred_1n = res1["y_pred_final"]

res2 = guard_and_mitigate_if_needed(
    pipeline=pipeline_2, X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
    thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
)
y_pred_2n = res2["y_pred_final"]

res3 = guard_and_mitigate_if_needed(
    pipeline=pipeline_3, X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
    thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
)
y_pred_3n = res3["y_pred_final"]


res4 = guard_and_mitigate_if_needed(
    pipeline=pipeline_4, X_train=X_train, y_train=y_train,
    X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
    thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
)
y_pred_4n = res4["y_pred_final"]


In [ ]:
A1 = accuracy_score(y_test, y_pred_1)
A2 = accuracy_score(y_test, y_pred_2)
A3 = accuracy_score(y_test, y_pred_3)
A4 = accuracy_score(y_test, y_pred_4)

f1 = f1_score(y_test, y_pred_1)
f2 = f1_score(y_test, y_pred_2)
f3 = f1_score(y_test, y_pred_3)
f4 = f1_score(y_test, y_pred_4)

In [ ]:
A1n = accuracy_score(y_test, y_pred_1n)
A2n = accuracy_score(y_test, y_pred_2n)
A3n = accuracy_score(y_test, y_pred_3n)
A4n = accuracy_score(y_test, y_pred_4n)

f1n = f1_score(y_test, y_pred_1n)
f2n = f1_score(y_test, y_pred_2n)
f3n = f1_score(y_test, y_pred_3n)
f4n = f1_score(y_test, y_pred_4n)

A1, A1n, A2, A2n, A3, A3n , A4,A4n

In [ ]:
from sklearn.dummy import DummyClassifier

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, stratify=X['age'])

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

def calculate_biases(mispred_p1, mispred_p2, mispred_p3,mispred_p4, mispred_baseline):
    """
    Calculate individual biases and total bias.

    Parameters:
    mispred_p1 (int): Mispredictions for P1 (LE+SS+Classifier)
    mispred_p2 (int): Mispredictions for P2 (SS+Classifier)
    mispred_p3 (int): Mispredictions for P3 (LE+Classifier)
    mispred_baseline (int): Mispredictions for the baseline model

    Returns:
    dict: A dictionary containing the individual biases and total bias
    """
    # Calculating individual biases
    BD = mispred_baseline
    BLE = mispred_p1 - mispred_p2
    LE_SS_Interaction = mispred_p1 - (mispred_p2 + mispred_p3 - mispred_baseline)

    BSS = mispred_p1 - mispred_p3
    SS_PC_Interaction = mispred_p1 - (mispred_p2 + mispred_p4 - mispred_baseline)
    
    BPC = mispred_p1 - mispred_p4
    PC_Classifier_Interaction = mispred_p1 - (mispred_p3 + mispred_p4 - mispred_baseline)

    # Calculate BC to ensure TB equals mispred_p1
    BC = mispred_p1 - (BD + BLE + BSS + BPC + LE_SS_Interaction + SS_PC_Interaction + PC_Classifier_Interaction)

    # Calculate the total bias
    TB = BD + BLE + BSS + BPC + BC + LE_SS_Interaction + SS_PC_Interaction + PC_Classifier_Interaction

    # Returning the results as a dictionary
    return {
        'BD': BD,

        'BSt-1': BLE,
        'St1-St2_Interaction': LE_SS_Interaction,
        'BSt-2': BSS,
        'St2-St3_Interaction': SS_PC_Interaction,
        'BSt-3': BPC,
              
        'St3_Classifier_Interaction': PC_Classifier_Interaction,
        'BC': BC,
        'TB': TB
    }

import matplotlib.pyplot as plt

def plot_biases(biases, mispred_p1):
    """
    Plot biases and observed mispredictions.
    """
    labels = [
        'Data (BD)', 
        
        'Preprocessing Stage 1', 
        'Stage 1-2 Interaction',
        'Preprocessing Stage 2', 
         
       
        'Stage 2-3 Interaction',
        'Preprocessing Stage 3',
        'Stage 3-Classifier Interaction',
        'Classifier (BC)', 
        'Total Bias (TB)', 
        'Observed Bias (mispred_p1)'
    ]
    
    values = [
        biases['BD'], 
         
        biases['BSt-1'],
        biases['St1-St2_Interaction'],
        biases['BSt-2'], 
        
        
        biases['BSt-3'],
        biases['St2-St3_Interaction'], 
        biases['St3_Classifier_Interaction'], 
        biases['BC'],
        biases['TB'], 
        mispred_p1
    ]

    colors = [
        'blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray'
    ]

    plt.figure(figsize=(12, 8))
    bars = plt.barh(labels, values, color=colors)
    plt.xlabel('Bias / Mispredictions')
    plt.title('Bias Analysis and Observed Mispredictions - After Mitigation')

    # Adding annotations
    for bar in bars:
        width = bar.get_width()
        label_x_pos = width if width < 0 else width
        plt.text(
            label_x_pos, bar.get_y() + bar.get_height() / 2,
            f'{width:.2f}', ha='center', va='center',
            bbox=dict(facecolor='white', alpha=0.7)
        )

    plt.axvline(0, color='black', linewidth=0.8)
    plt.xlim(min(values) - 10, max(values) + 10)  # Adding some padding to the x-axis
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    plt.show()

# Example usage with hypothetical values



# Assuming X_train, y_train, X_test, y_test, y_pred_1, y_pred_2, y_pred_3 are defined
mispred_p1 = sum(y_test != y_pred_1n)
mispred_p2 = sum(y_test != y_pred_2n)
mispred_p3 = sum(y_test != y_pred_3n)
mispred_p4 = sum(y_test != y_pred_4n)



# biases = calculate_biases_multiple(mispred_p1, mispred_p2, mispred_p3, mispred_p4, mispred_baseline)

# Calculate biases
biases = calculate_biases(mispred_p1, mispred_p2, mispred_p3,mispred_p4, mispred_baseline)
plot_biases(biases, mispred_p1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize volatility arrays
volatility_LE = np.zeros(len(y_test))
volatility_SS = np.zeros(len(y_test))
volatility_classifier = np.zeros(len(y_test))

def calculate_volatility(y_test, y_pred1, y_pred2):
    volatility = np.zeros(len(y_test))
    for i in range(len(y_test)):
        if y_pred1[i] == y_test.iloc[i] and y_pred2[i] == y_test.iloc[i]:
            volatility[i] = 0
        elif y_pred1[i] == y_test.iloc[i] and y_pred2[i] != y_test.iloc[i]:
            volatility[i] = 1
        elif y_pred1[i] != y_test.iloc[i] and y_pred2[i] == y_test.iloc[i]:
            volatility[i] = 2
        elif y_pred1[i] != y_test.iloc[i] and y_pred2[i] != y_test.iloc[i] and y_pred1[i] == y_pred2[i]:
            volatility[i] = 3
        elif y_pred1[i] != y_test.iloc[i] and y_pred2[i] != y_test.iloc[i] and y_pred1[i] != y_pred2[i]:
            volatility[i] = 4
    return volatility

# Calculate volatility scores
volatility_LE = calculate_volatility(y_test, y_pred_1n, y_pred_2n)
volatility_SS = calculate_volatility(y_test, y_pred_1n, y_pred_3n)
volatility_classifier = calculate_volatility(y_test, y_pred_1n, y_pred_baseline)
volatility_y_pred_4n = np.zeros(len(y_test))

# Calculate volatility for y_pred_4n
volatility_y_pred_4n = calculate_volatility(y_test, y_pred_1n, y_pred_4n)

# Update the summary with y_pred_4n values
volatility_summary = {
    'Condition': [
        'Vol_PS1=0', 'Vol_PS1=1', 'Vol_PS1=2', 'Vol_PS1=3', 'Vol_PS1=4',
        'Vol_PS2=0', 'Vol_PS2=1', 'Vol_PS2=2', 'Vol_PS2=3', 'Vol_PS2=4',
        'Vol_Cl=0', 'Vol_Cl=1', 'Vol_Cl=2', 'Vol_Cl=3', 'Vol_Cl=4',
        'Vol_P4=0', 'Vol_P4=1', 'Vol_P4=2', 'Vol_P4=3', 'Vol_P4=4'
    ],
    'Count': [
        np.sum(volatility_LE == 0), np.sum(volatility_LE == 1), np.sum(volatility_LE == 2), np.sum(volatility_LE == 3), np.sum(volatility_LE == 4),
        np.sum(volatility_SS == 0), np.sum(volatility_SS == 1), np.sum(volatility_SS == 2), np.sum(volatility_SS == 3), np.sum(volatility_SS == 4),
        np.sum(volatility_classifier == 0), np.sum(volatility_classifier == 1), np.sum(volatility_classifier == 2), np.sum(volatility_classifier == 3), np.sum(volatility_classifier == 4),
        np.sum(volatility_y_pred_4n == 0), np.sum(volatility_y_pred_4n == 1), np.sum(volatility_y_pred_4n == 2), np.sum(volatility_y_pred_4n == 3), np.sum(volatility_y_pred_4n == 4)
    ]
}

# Create a DataFrame
df_volatility = pd.DataFrame(volatility_summary)

# Plotting
plt.figure(figsize=(16, 8))
ax = sns.barplot(x='Condition', y='Count', data=df_volatility, palette='viridis')

plt.title('Volatility Score Analysis - After Mitigation ')
plt.xlabel('Condition')
plt.ylabel('Count')
for i in ax.containers:
    ax.bar_label(i, fmt='%.0f', label_type='edge')

plt.show()

In [ ]:
207+1144

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score


# Example computed metrics for illustration
accuracy_1= A1
accuracy_2 = np.abs(A1- A2)
accuracy_3 = np.abs(A1- A3)
accuracy_4 = np.abs(A1- A4)

accuracy_1n= A1n
accuracy_2n= np.abs(A1n- A2n)
accuracy_3n = np.abs(A1n- A3n)
accuracy_4n = np.abs(A1n- A4n)

f1_1 = f1
f1_2 = np.abs(f1-f2)
f1_3 = np.abs(f1-f3)
f1_4 = np.abs(f1-f4)

f1_1n = f1n
f1_2n = np.abs(f1n-f2n)
f1_3n = np.abs(f1n-f3n)
f1_4n = np.abs(f1n-f4n)


# Function to plot the accuracy and F1 scores for pipelines and components before and after mitigation
def plot_metrics():
    labels = ['Pipeline', 'SS Component' , 'Custom FE Component' ,'Stratify Component']  # Update this if needed for component names
    
    # Accuracy plot
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    before_mitigation_accuracy = [accuracy_1, accuracy_2, accuracy_3, accuracy_4]
    after_mitigation_accuracy = [accuracy_1n, accuracy_2n,accuracy_3n, accuracy_3n]
    
    bar_width = 0.35  # Width of the bars
    x = np.arange(len(labels))
    
    plt.bar(x - bar_width / 2, before_mitigation_accuracy, width=bar_width, label='Before Mitigation', color='blue')
    plt.bar(x + bar_width / 2, after_mitigation_accuracy, width=bar_width, label='After Mitigation', color='green')
    
    plt.xticks(x, labels)
    plt.ylabel('Accuracy')
    plt.title('Accuracy Before vs After Mitigation')
    plt.xticks(rotation=70)

    plt.legend()

    # F1 Score plot
    plt.subplot(1, 2, 2)
    before_mitigation_f1 = [f1_1, f1_2,f1_3, f1_4]
    after_mitigation_f1 = [f1_1n, f1_2n,f1_3n , f1_4n]
    
    plt.bar(x - bar_width / 2, before_mitigation_f1, width=bar_width, label='Before Mitigation', color='blue')
    plt.bar(x + bar_width / 2, after_mitigation_f1, width=bar_width, label='After Mitigation', color='green')
    
    plt.xticks(x, labels)
    plt.ylabel('F1 Score')
    plt.title('F1 Score Before vs After Mitigation')
    plt.xticks(rotation=70)

    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage
plot_metrics()


In [ ]:
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

spd_cf_list = []
eod_cf_list = []
aod_cf_list = []
erd_cf_list = []

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    
    # Convert to dense if X is sparse
    if isinstance(X, csr_matrix):
        X = X.toarray()
    
    X_imputed = imputer.fit_transform(X)
    
    # Use the provided treatment column directly
    treatment = np.where((treatment_column == 4) | (treatment_column == 'White'), 1, 0)

    # Use the imputed covariate matrix without the treatment column
    X_covariates = X_imputed

    # One-hot encode the covariate matrix (if needed)
    X_encoded = pd.get_dummies(pd.DataFrame(X_covariates), drop_first=True).values

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0]  # 'White' or 4 are considered treated
    control_indices = np.where((treatment == 0))[0]  # All others are control

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices

def flip_race(attribute):
    return ['Non-White' if x == '4' else 'White' for x in attribute]

# Loading and preprocessing dataset as in your provided code
train_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.data'
test_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.test'

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
                'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']
na_values = ['?']

train = pd.read_csv(train_path, header=None, names=column_names, 
                    skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=None, names=column_names,
                   skipinitialspace=True, na_values=na_values)

train['income-per-year'] = train['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
test['income-per-year'] = test['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

combined_data = pd.concat([train, test])
combined_data = combined_data.dropna(subset=['income-per-year'])

X_combined = combined_data.drop('income-per-year', axis=1)
y_combined = combined_data['income-per-year']

categorical_cols = X_combined.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_combined.select_dtypes(include=['int64', 'float64']).columns.tolist()

X_combined[categorical_cols] = X_combined[categorical_cols].astype(str)
X_combined['age'] = pd.to_numeric(X_combined['age'], errors='coerce')

# Custom transformers

# Main loop with updated function usage
for i in range(10):
    print(f"Iteration {i+1}")
    X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42, stratify=y_combined)
    X_train_no_strat, X_test_no_strat, y_train_no_strat, y_test_no_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42)

    pipeline_1.fit(X_train_strat, y_train_strat)
    pipeline_2.fit(X_train_strat, y_train_strat)
    pipeline_3.fit(X_train_strat, y_train_strat)

    # For P4
    pipeline_4.fit(X_train_no_strat, y_train_no_strat)

    y_pred_1 = pipeline_1.predict(X_test_strat)
    y_pred_2 = pipeline_2.predict(X_test_strat)
    y_pred_3 = pipeline_3.predict(X_test_strat)

# For P4
    y_pred_4 = pipeline_4.predict(X_test_no_strat)
    # Placeholder for Fairness_Metrics_Computation function
    # Assuming it returns the necessary SPD, EOD, AOD, ERD metrics
    SPD_mv, EOD_mv, AOD_mv, ERD_mv = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['race'])
    SPD_fe, EOD_fe, AOD_fe, ERD_fe = Fairness_Metrics_Computation(y_pred_1, y_pred_3, y_test, X_test['race']) 
    SPD_3, EOD_3, AOD_3, ERD_3 = Fairness_Metrics_Computation(y_pred_1, y_pred_4, y_test, X_test['race'])
    
   
    spd_list.append((SPD_mv, SPD_fe,SPD_3))
    eod_list.append((EOD_mv, EOD_fe,EOD_3))
    aod_list.append((AOD_mv, AOD_fe,AOD_3))
    erd_list.append((ERD_mv, ERD_fe, ERD_3))

    X_test_flipped = X_test.copy()
    X_test_flipped['race'] = flip_race(X_test['race'])

    X_test_flipped_transformed1 = pipeline_1.named_steps['preprocessor'].transform(X_test_flipped)
    X_test_flipped_transformed2 = pipeline_2.named_steps['preprocessor'].transform(X_test_flipped)
    X_test_flipped_transformed3 = pipeline_3.named_steps['preprocessor'].transform(X_test_flipped)
    X_test_flipped_transformed4 = pipeline_4.named_steps['preprocessor'].transform(X_test_flipped)

    y_pred_cf_1 = pipeline_1.named_steps['classifier'].predict(X_test_flipped_transformed1)
    y_pred_cf_2 = pipeline_2.named_steps['classifier'].predict(X_test_flipped_transformed2)
    y_pred_cf_3 = pipeline_3.named_steps['classifier'].predict(X_test_flipped_transformed3)
    y_pred_cf_4 = pipeline_3.named_steps['classifier'].predict(X_test_flipped_transformed3)

    
    SPD_cf_mv, EOD_cf_mv, AOD_cf_mv, ERD_cf_mv = Fairness_Metrics_Computation(y_pred_cf_1, y_pred_cf_2, y_test, X_test['race'])
    SPD_cf_fe, EOD_cf_fe, AOD_cf_fe, ERD_cf_fe = Fairness_Metrics_Computation(y_pred_cf_1, y_pred_cf_3, y_test, X_test['race'])
    SPD_cf_3, EOD_cf_3, AOD_cf_3, ERD_cf_3 = Fairness_Metrics_Computation(y_pred_cf_1, y_pred_cf_4, y_test, X_test['race'])

    spd_cf_list.append((SPD_cf_mv, SPD_cf_fe, SPD_cf_3))
    eod_cf_list.append((EOD_cf_mv, EOD_cf_fe,EOD_cf_3))
    aod_cf_list.append((AOD_cf_mv, AOD_cf_fe,AOD_cf_3))
    erd_cf_list.append((ERD_cf_mv, ERD_cf_fe,ERD_cf_3))
    
    # Use the race column from X_test for computing propensity scores
    propensity_scores1, treatment1 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices1 = perform_matching(propensity_scores1, treatment1)

    propensity_scores2, treatment2 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices2 = perform_matching(propensity_scores2, treatment2)

    propensity_scores3, treatment3 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices3 = perform_matching(propensity_scores3, treatment3)

    propensity_scores4, treatment4 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices4 = perform_matching(propensity_scores4, treatment4)
    
    common_matched_indices = set(matched_indices1).intersection(matched_indices2).intersection(matched_indices3).intersection(matched_indices4)
    common_matched_indices = list(common_matched_indices)
    common_matched_indices.sort()

    y1_matched = y_pred_1[common_matched_indices]
    y2_matched = y_pred_2[common_matched_indices]
    y3_matched = y_pred_3[common_matched_indices]
    y4_matched = y_pred_4[common_matched_indices]
    
    y_test_matched = y_test.reset_index(drop=True)[common_matched_indices]
    sex_matched = X_test['race'].reset_index(drop=True)[common_matched_indices]

    SPD_cas_mv, EOD_cas_mv, AOD_cas_mv, ERD_cas_mv = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched)
    SPD_cas_fe, EOD_cas_fe, AOD_cas_fe, ERD_cas_fe = Fairness_Metrics_Computation(y1_matched, y3_matched, y_test_matched, sex_matched)
    SPD_cas_3, EOD_cas_3, AOD_cas_3, ERD_cas_3 = Fairness_Metrics_Computation(y1_matched, y4_matched, y_test_matched, sex_matched)

    spd_cas_list.append((SPD_cas_mv, SPD_cas_fe,SPD_cas_3))
    eod_cas_list.append((EOD_cas_mv, EOD_cas_fe,EOD_cas_3))
    aod_cas_list.append((AOD_cas_mv, AOD_cas_fe,AOD_cas_3))
    erd_cas_list.append((ERD_cas_mv, ERD_cas_fe,ERD_cas_3))



In [ ]:
spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

spd_cf_array = np.array(spd_cf_list)
eod_cf_array = np.array(eod_cf_list)
aod_cf_array = np.array(aod_cf_list)
erd_cf_array = np.array(erd_cf_list)


spd_mean_mv = spd_array[:, 0].mean()
spd_mean_fe = spd_array[:, 1].mean()
spd_mean_enc = spd_array[:, 2].mean()

eod_mean_mv = eod_array[:, 0].mean()
eod_mean_fe = eod_array[:, 1].mean()
eod_mean_enc = eod_array[:, 2].mean()

aod_mean_mv = aod_array[:, 0].mean()
aod_mean_fe = aod_array[:, 1].mean()
aod_mean_enc = aod_array[:, 2].mean()

erd_mean_mv = erd_array[:, 0].mean()
erd_mean_fe = erd_array[:, 1].mean()
erd_mean_enc = erd_array[:, 2].mean()

spd_cas_mean_mv = spd_cas_array[:, 0].mean()
spd_cas_mean_fe = spd_cas_array[:, 1].mean()
spd_cas_mean_enc = spd_cas_array[:, 2].mean()

eod_cas_mean_mv = eod_cas_array[:, 0].mean()
eod_cas_mean_fe = eod_cas_array[:, 1].mean()
eod_cas_mean_enc = eod_cas_array[:, 2].mean()

aod_cas_mean_mv = aod_cas_array[:, 0].mean()
aod_cas_mean_fe = aod_cas_array[:, 1].mean()
aod_cas_mean_enc = aod_cas_array[:, 2].mean()

erd_cas_mean_mv = erd_cas_array[:, 0].mean()
erd_cas_mean_fe = erd_cas_array[:, 1].mean()
erd_cas_mean_enc = erd_cas_array[:, 2].mean()

spd_cf_mean_mv = spd_cf_array[:, 0].mean()
spd_cf_mean_fe = spd_cf_array[:, 1].mean()
spd_cf_mean_enc = spd_cf_array[:, 2].mean()

eod_cf_mean_mv = eod_cf_array[:, 0].mean()
eod_cf_mean_fe = eod_cf_array[:, 1].mean()
eod_cf_mean_enc = eod_cf_array[:, 2].mean()

aod_cf_mean_mv = aod_cf_array[:, 0].mean()
aod_cf_mean_fe = aod_cf_array[:, 1].mean()
aod_cf_mean_enc = aod_cf_array[:, 2].mean()

erd_cf_mean_mv = erd_cf_array[:, 0].mean()
erd_cf_mean_fe = erd_cf_array[:, 1].mean()
erd_cf_mean_enc = erd_cf_array[:, 2].mean()


spd_mean = spd_array.mean(axis=0)
eod_mean = eod_array.mean(axis=0)
aod_mean = aod_array.mean(axis=0)
erd_mean = erd_array.mean(axis=0)

spd_cas_mean = spd_cas_array.mean(axis=0)
eod_cas_mean = eod_cas_array.mean(axis=0)
aod_cas_mean = aod_cas_array.mean(axis=0)
erd_cas_mean = erd_cas_array.mean(axis=0)

spd_cf_mean = spd_cf_array.mean(axis=0)
eod_cf_mean = eod_cf_array.mean(axis=0)
aod_cf_mean = aod_cf_array.mean(axis=0)
erd_cf_mean = erd_cf_array.mean(axis=0)

# Calculate the standard errors for each metric and stage
spd_se = spd_array.std(axis=0) / np.sqrt(spd_array.shape[0])
eod_se = eod_array.std(axis=0) / np.sqrt(eod_array.shape[0])
aod_se = aod_array.std(axis=0) / np.sqrt(aod_array.shape[0])
erd_se = erd_array.std(axis=0) / np.sqrt(erd_array.shape[0])

spd_cas_se = spd_cas_array.std(axis=0) / np.sqrt(spd_cas_array.shape[0])
eod_cas_se = eod_cas_array.std(axis=0) / np.sqrt(eod_cas_array.shape[0])
aod_cas_se = aod_cas_array.std(axis=0) / np.sqrt(aod_cas_array.shape[0])
erd_cas_se = erd_cas_array.std(axis=0) / np.sqrt(erd_cas_array.shape[0])

spd_cf_se = spd_cf_array.std(axis=0) / np.sqrt(spd_cf_array.shape[0])
eod_cf_se = eod_cf_array.std(axis=0) / np.sqrt(eod_cf_array.shape[0])
aod_cf_se = aod_cf_array.std(axis=0) / np.sqrt(aod_cf_array.shape[0])
erd_cf_se = erd_cf_array.std(axis=0) / np.sqrt(erd_cf_array.shape[0])

# Plotting function
def plot_with_error_bars(ax, means, std_errors, title, metric_labels, colors, labels):
    bar_width = 0.25
    indices = np.arange(len(metric_labels))  # Fix this to use metric_labels length
    
    for i, (mean, std_error, color, label) in enumerate(zip(means, std_errors, colors, labels)):
        ax.bar(indices + i * bar_width, mean, yerr=std_error, capsize=5, width=bar_width, align='center', color=color, label=label)
    
    ax.set_xticks(indices + bar_width)
    ax.set_xticklabels(metric_labels)
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.legend()

# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Create subplots
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Colors and labels for the bars
colors = ['blue', 'orange', 'green']
labels = ['Statistical', 'Causal', 'Counterfactual']

# Prepare means and standard errors for plotting
means_mv = [spd_mean[0], eod_mean[0], aod_mean[0], erd_mean[0]]
std_errors_mv = [spd_se[0], eod_se[0], aod_se[0], erd_se[0]]
means_cas_mv = [spd_cas_mean[0], eod_cas_mean[0], aod_cas_mean[0], erd_cas_mean[0]]
std_errors_cas_mv = [spd_cas_se[0], eod_cas_se[0], aod_cas_se[0], erd_cas_se[0]]
means_cf_mv = [spd_cf_mean[0], eod_cf_mean[0], aod_cf_mean[0], erd_cf_mean[0]]
std_errors_cf_mv = [spd_cf_se[0], eod_cf_se[0], aod_cf_se[0], erd_cf_se[0]]

# Plot for MV
plot_with_error_bars(axs[0], [means_mv, means_cas_mv, means_cf_mv], [std_errors_mv, std_errors_cas_mv, std_errors_cf_mv], 'SS', metric_labels, colors, labels)

# Prepare means and standard errors for plotting FE
means_fe = [spd_mean[1], eod_mean[1], aod_mean[1], erd_mean[1]]
std_errors_fe = [spd_se[1], eod_se[1], aod_se[1], erd_se[1]]
means_cas_fe = [spd_cas_mean[1], eod_cas_mean[1], aod_cas_mean[1], erd_cas_mean[1]]
std_errors_cas_fe = [spd_cas_se[1], eod_cas_se[1], aod_cas_se[1], erd_cas_se[1]]
means_cf_fe = [spd_cf_mean[1], eod_cf_mean[1], aod_cf_mean[1], erd_cf_mean[1]]
std_errors_cf_fe = [spd_cf_se[1], eod_cf_se[1], aod_cf_se[1], erd_cf_se[1]]

# Plot for FE
plot_with_error_bars(axs[1], [means_fe, means_cas_fe, means_cf_fe], [std_errors_fe, std_errors_cas_fe, std_errors_cf_fe], 'Custom FE', metric_labels, colors, labels)

# Prepare means and standard errors for plotting Encoding
means_enc = [spd_mean[2], eod_mean[2], aod_mean[2], erd_mean[2]]
std_errors_enc = [spd_se[2], eod_se[2], aod_se[2], erd_se[2]]
means_cas_enc = [spd_cas_mean[2], eod_cas_mean[2], aod_cas_mean[2], erd_cas_mean[2]]
std_errors_cas_enc = [spd_cas_se[2], eod_cas_se[2], aod_cas_se[2], erd_cas_se[2]]
means_cf_enc = [spd_cf_mean[2], eod_cf_mean[2], aod_cf_mean[2], erd_cf_mean[2]]
std_errors_cf_enc = [spd_cf_se[2], eod_cf_se[2], aod_cf_se[2], erd_cf_se[2]]

# Plot for Encoding
plot_with_error_bars(axs[2], [means_enc, means_cas_enc, means_cf_enc], [std_errors_enc, std_errors_cas_enc, std_errors_cf_enc], 'Stratify', metric_labels, colors, labels)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

def Fairness_Metrics_Computation(y1, y2, y, attribute, compute_type='global'):
    """
    Compute local or global fairness metrics: SPD, EOD, AOD, ERD
    y1: Predictions from original pipeline (Y(P))
    y2: Predictions from modified pipeline (Y(P*))
    y: Ground truth labels
    attribute: Sensitive attribute (e.g., race or gender)
    compute_type: 'local' for local metrics (only instances where y1 != y2),
                  'global' for global metrics (all instances)
    """
    # Convert inputs to pandas Series for easier handling
    y1 = pd.Series(y1)
    y2 = pd.Series(y2)
    y = pd.Series(y)
    attribute = pd.Series(attribute)

    # Identify privileged and unprivileged groups
    privileged = (attribute == 'White') | (attribute == 1)
    unprivileged = ~privileged

    # Set conditions for local or global calculation
    condition = (y1 != y2) if compute_type == 'local' else (y1 == y1)  # y1 == y1 includes all rows

    # Counts of privileged and unprivileged groups under the selected condition
    count_privileged = np.sum(privileged & condition)
    count_unprivileged = np.sum(unprivileged & condition)

    # SPD Calculation
    SFC_SPD_P = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                        for y1_i, y2_i, attr, cond in zip(y1, y2, attribute, condition) if (attr == 'White' or attr == 1) and cond])
    SFC_SPD_NP = np.sum([(1 if y1_i == 1 and y2_i == 0 else -1 if y1_i == 0 and y2_i == 1 else 0)
                         for y1_i, y2_i, attr, cond in zip(y1, y2, attribute, condition) if (attr != 'White' and attr != 1) and cond])

    SFR_SPD_p = SFC_SPD_P / count_privileged if count_privileged > 0 else 0
    SFR_SPD_np = SFC_SPD_NP / count_unprivileged if count_unprivileged > 0 else 0
    SPD = SFR_SPD_np - SFR_SPD_p

    # EOD Calculation
    P_Y1 = np.sum((y == 1) & privileged & condition)
    NP_Y1 = np.sum((y == 1) & unprivileged & condition)

    SFC_EOD_P = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr == 'White' or attr == 1) and cond])
    SFC_EOD_NP = np.sum([(1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 0)
                         for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr != 'White' and attr != 1) and cond])

    SFR_EOD_p = SFC_EOD_P / P_Y1 if P_Y1 > 0 else 0
    SFR_EOD_np = SFC_EOD_NP / NP_Y1 if NP_Y1 > 0 else 0
    EOD = SFR_EOD_np - SFR_EOD_p

    # AOD Calculation
    P_Y0 = np.sum((y == 0) & privileged & condition)
    NP_Y0 = np.sum((y == 0) & unprivileged & condition)

    SFC_TP_P = SFC_EOD_P  # True Positives for privileged group
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr == 'White' or attr == 1) and cond])
    SFC_TP_NP = SFC_EOD_NP  # True Positives for unprivileged group
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr != 'White' and attr != 1) and cond])

    SFR_AOD_p = (SFC_TP_P / P_Y1 if P_Y1 > 0 else 0) + (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0)
    SFR_AOD_np = (SFC_TP_NP / NP_Y1 if NP_Y1 > 0 else 0) + (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0)
    AOD = (SFR_AOD_np - SFR_AOD_p) / 2

    # ERD Calculation
    SFC_FN_P = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                       for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr == 'White' or attr == 1) and cond])
    SFC_FN_NP = np.sum([(-1 if y_true_i == 1 and y1_i == 0 and y2_i == 1 else 1 if y_true_i == 1 and y1_i == 1 and y2_i == 0 else 0)
                        for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr != 'White' and attr != 1) and cond])
    SFC_FP_P = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                       for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr == 'White' or attr == 1) and cond])
    SFC_FP_NP = np.sum([(1 if y_true_i == 0 and y1_i == 1 and y2_i == 0 else -1 if y_true_i == 0 and y1_i == 0 and y2_i == 1 else 0)
                        for y_true_i, y1_i, y2_i, attr, cond in zip(y, y1, y2, attribute, condition) if (attr != 'White' and attr != 1) and cond])

    SFR_ERR_p = (SFC_FP_P / P_Y0 if P_Y0 > 0 else 0) + (SFC_FN_P / P_Y1 if P_Y1 > 0 else 0)
    SFR_ERR_np = (SFC_FP_NP / NP_Y0 if NP_Y0 > 0 else 0) + (SFC_FN_NP / NP_Y1 if NP_Y1 > 0 else 0)
    ERD = SFR_ERR_np - SFR_ERR_p

    return SPD, EOD, AOD, ERD


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np



def flip_race(attribute):
    return ['Non-White' if x == '4' else 'White' for x in attribute]

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, handle_unknown='ignore'):
        self.columns = columns
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        self.encoders = {}
        if isinstance(X, pd.DataFrame):
            for col in self.columns:
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[col])
        else:
            for i, col in enumerate(self.columns):
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[:, i])
        return self

    def transform(self, X):
        X_copy = X.copy()
        if isinstance(X_copy, pd.DataFrame):
            for col in self.columns:
                X_copy[col] = X_copy[col].map(lambda s: self.encoders[col].transform([s])[0] 
                                              if s in self.encoders[col].classes_ else -1)
        else:
            for i, col in enumerate(self.columns):
                X_copy[:, i] = np.array([self.encoders[col].transform([s])[0] 
                                         if s in self.encoders[col].classes_ else -1 
                                         for s in X_copy[:, i]])
        return X_copy

# The rest of your code remains the same

# Paths to train and test datasets
train_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.data'
test_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.test'

# Define column names and missing values
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
                'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']
na_values = ['?']

# Read the train and test datasets
train = pd.read_csv(train_path, header=None, names=column_names, 
                    skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=None, names=column_names,
                   skipinitialspace=True, na_values=na_values)

# Map target variable
train['income-per-year'] = train['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
test['income-per-year'] = test['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

# Combine train and test data for re-splitting
combined_data = pd.concat([train, test])

# Drop rows with missing target values
combined_data = combined_data.dropna(subset=['income-per-year'])

X_combined = combined_data.drop('income-per-year', axis=1)
y_combined = combined_data['income-per-year']

# Categorical and numerical columns
categorical_cols = X_combined.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_combined.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Convert all categorical columns to strings to avoid type issues
X_combined[categorical_cols] = X_combined[categorical_cols].astype(str)

# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

gspd_list = []
geod_list = []
gaod_list = []
gerd_list = []

gspd_cas_list = []
geod_cas_list = []
gaod_cas_list = []
gerd_cas_list = []

# Perform 10 iterations
for i in range(10):
    print(f"Iteration {i+1}")
    # Randomly split the data into training and test sets
    X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42, stratify=y_combined)
    X_train_no_strat, X_test_no_strat, y_train_no_strat, y_test_no_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42)

    pipeline_1.fit(X_train_strat, y_train_strat)
    pipeline_2.fit(X_train_strat, y_train_strat)
    pipeline_3.fit(X_train_strat, y_train_strat)

    # For P4
    pipeline_4.fit(X_train_no_strat, y_train_no_strat)

    y_pred_1 = pipeline_1.predict(X_test_strat)
    y_pred_2 = pipeline_2.predict(X_test_strat)
    y_pred_3 = pipeline_3.predict(X_test_strat)

# For P4
    y_pred_4 = pipeline_4.predict(X_test_no_strat)
    
    SPD_1, EOD_1, AOD_1, ERD_1 = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['sex'],compute_type='local')
    SPD_2, EOD_2, AOD_2, ERD_2 = Fairness_Metrics_Computation(y_pred_1, y_pred_3, y_test, X_test['sex'],compute_type='local')
    SPD_3, EOD_3, AOD_3, ERD_3 = Fairness_Metrics_Computation(y_pred_1, y_pred_4, y_test, X_test['sex'],compute_type='local')
    
    # Store results
    spd_list.append((SPD_1, SPD_2, SPD_3))
    eod_list.append((EOD_1, EOD_2, EOD_3))
    aod_list.append((AOD_1, AOD_2, AOD_3))
    erd_list.append((ERD_1, ERD_2, ERD_3))
    
    gSPD_1, gEOD_1, gAOD_1, gERD_1 = Fairness_Metrics_Computation(y_pred_1, y_pred_2, y_test, X_test['sex'],compute_type='global')
    gSPD_2, gEOD_2, gAOD_2, gERD_2 = Fairness_Metrics_Computation(y_pred_1, y_pred_3, y_test, X_test['sex'],compute_type='global')
    gSPD_3, gEOD_3, gAOD_3, gERD_3 = Fairness_Metrics_Computation(y_pred_1, y_pred_4, y_test, X_test['sex'],compute_type='global')
    
    # Store results
    gspd_list.append((gSPD_1, gSPD_2, gSPD_3))
    geod_list.append((gEOD_1, gEOD_2, gEOD_3))
    gaod_list.append((gAOD_1, gAOD_2, gAOD_3))
    gerd_list.append((gERD_1, gERD_2, gERD_3))
       
    propensity_scores1, treatment1 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices1 = perform_matching(propensity_scores1, treatment1)

    propensity_scores2, treatment2 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices2 = perform_matching(propensity_scores2, treatment2)

    propensity_scores3, treatment3 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices3 = perform_matching(propensity_scores3, treatment3)

    propensity_scores4, treatment4 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices4 = perform_matching(propensity_scores4, treatment4)
    
    common_matched_indices = set(matched_indices1).intersection(matched_indices2).intersection(matched_indices3).intersection(matched_indices4)
    common_matched_indices = list(common_matched_indices)
    common_matched_indices.sort()

    y1_matched = y_pred_1[common_matched_indices]
    y2_matched = y_pred_2[common_matched_indices]
    y3_matched = y_pred_3[common_matched_indices]
    y4_matched = y_pred_4[common_matched_indices]
    
    y_test_matched = y_test.reset_index(drop=True)[common_matched_indices]
    sex_matched = X_test['race'].reset_index(drop=True)[common_matched_indices]
    
    # Compute causal fairness metrics
    SPD_1_cas, EOD_1_cas, AOD_1_cas, ERD_1_cas = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched ,compute_type='local')
    SPD_2_cas, EOD_2_cas, AOD_2_cas, ERD_2_cas = Fairness_Metrics_Computation(y1_matched, y3_matched, y_test_matched, sex_matched,compute_type='local')
    SPD_3_cas, EOD_3_cas, AOD_3_cas, ERD_3_cas = Fairness_Metrics_Computation(y1_matched, y4_matched, y_test_matched, sex_matched,compute_type='local')
    
    # Store causal results
    spd_cas_list.append((SPD_1_cas, SPD_2_cas, SPD_3_cas))
    eod_cas_list.append((EOD_1_cas, EOD_2_cas, EOD_3_cas))
    aod_cas_list.append((AOD_1_cas, AOD_2_cas, AOD_3_cas))
    erd_cas_list.append((ERD_1_cas, ERD_2_cas, ERD_3_cas))

    gSPD_1_cas, gEOD_1_cas, gAOD_1_cas, gERD_1_cas = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched ,compute_type='global')
    gSPD_2_cas, gEOD_2_cas, gAOD_2_cas, gERD_2_cas = Fairness_Metrics_Computation(y1_matched, y3_matched, y_test_matched, sex_matched,compute_type='global')
    gSPD_3_cas, gEOD_3_cas, gAOD_3_cas, gERD_3_cas = Fairness_Metrics_Computation(y1_matched, y4_matched, y_test_matched, sex_matched,compute_type='global')
    
    # Store causal results
    gspd_cas_list.append((gSPD_1_cas, gSPD_2_cas, gSPD_3_cas))
    geod_cas_list.append((gEOD_1_cas, gEOD_2_cas, gEOD_3_cas))
    gaod_cas_list.append((gAOD_1_cas, gAOD_2_cas, gAOD_3_cas))
    gerd_cas_list.append((gERD_1_cas, gERD_2_cas, gERD_3_cas))
    


In [ ]:
def compute_means_and_errors(array):
    mean = array.mean(axis=0) * 0.1
    se = (array.std(axis=0) / np.sqrt(array.shape[0])) * 0.1
    return np.round(mean, 3), np.round(se, 3)


spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

gspd_array = np.array(gspd_list)
geod_array = np.array(geod_list)
gaod_array = np.array(gaod_list)
gerd_array = np.array(gerd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

gspd_cas_array = np.array(gspd_cas_list)
geod_cas_array = np.array(geod_cas_list)
gaod_cas_array = np.array(gaod_cas_list)
gerd_cas_array = np.array(gerd_cas_list)


spd_mean, spd_se = compute_means_and_errors(spd_array)
eod_mean, eod_se = compute_means_and_errors(eod_array)
aod_mean, aod_se = compute_means_and_errors(aod_array)
erd_mean, erd_se = compute_means_and_errors(erd_array)

gspd_mean, gspd_se = compute_means_and_errors(gspd_array)
geod_mean, geod_se = compute_means_and_errors(geod_array)
gaod_mean, gaod_se = compute_means_and_errors(gaod_array)
gerd_mean, gerd_se = compute_means_and_errors(gerd_array)

spd_cas_mean, spd_cas_se = compute_means_and_errors(spd_cas_array)
eod_cas_mean, eod_cas_se = compute_means_and_errors(eod_cas_array)
aod_cas_mean, aod_cas_se = compute_means_and_errors(aod_cas_array)
erd_cas_mean, erd_cas_se = compute_means_and_errors(erd_cas_array)

gspd_cas_mean, gspd_cas_se = compute_means_and_errors(gspd_cas_array)
geod_cas_mean, geod_cas_se = compute_means_and_errors(geod_cas_array)
gaod_cas_mean, gaod_cas_se = compute_means_and_errors(gaod_cas_array)
gerd_cas_mean, gerd_cas_se = compute_means_and_errors(gerd_cas_array)


import numpy as np
import matplotlib.pyplot as plt

# Assuming your fairness metrics arrays are correctly formatted
# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Colors for the bars
colors = ['blue', 'orange']
labels = ['Local', 'Global']

# Plot function for fairness metrics
def plot_fairness_metrics(ax, local_means, global_means, local_se, global_se, title, metric_labels):
    x = np.arange(len(metric_labels))  # label locations
    width = 0.35  # the width of the bars

    rects1 = ax.bar(x - width/2, local_means, width, label='Local', yerr=local_se, capsize=5)
    rects2 = ax.bar(x + width/2, global_means, width, label='Global', yerr=global_se, capsize=5)

    # Add labels, title, and custom ticks
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(metric_labels)
    ax.legend()

# Replace these with the actual means and errors for statistical and causal fairness
# Statistical Local and Global for PCA
local_mean_stage_1 = [spd_mean[0], eod_mean[0], aod_mean[0], erd_mean[0]]
local_se_stage_1 = [spd_se[0], eod_se[0], aod_se[0], erd_se[0]]

global_mean_stage_1 = [gspd_mean[0], geod_mean[0], gaod_mean[0], gerd_mean[0]]
global_se_stage_1 = [gspd_se[0], geod_se[0], gaod_se[0], gerd_se[0]]

# Causal Local and Global for PCA
causal_means_stage_1 = [spd_cas_mean[0], eod_cas_mean[0], aod_cas_mean[0], erd_cas_mean[0]]
causal_se_stage_1 = [spd_cas_se[0], eod_cas_se[0], aod_cas_se[0], erd_cas_se[0]]

gcausal_means_stage_1 = [gspd_cas_mean[0], geod_cas_mean[0], gaod_cas_mean[0], gerd_cas_mean[0]]
gcausal_se_stage_1 = [gspd_cas_se[0], geod_cas_se[0], gaod_cas_se[0], gerd_cas_se[0]]

# Statistical Local and Global for SelectKBest
local_mean_stage_2 = [spd_mean[1], eod_mean[1], aod_mean[1], erd_mean[1]]
local_se_stage_2 = [spd_se[1], eod_se[1], aod_se[1], erd_se[1]]

global_mean_stage_2 = [gspd_mean[1], geod_mean[1], gaod_mean[1], gerd_mean[1]]
global_se_stage_2 = [gspd_se[1], geod_se[1], gaod_se[1], gerd_se[1]]

# Causal Local and Global for SelectKBest
causal_means_stage_2 = [spd_cas_mean[1], eod_cas_mean[1], aod_cas_mean[1], erd_cas_mean[1]]
causal_se_stage_2 = [spd_cas_se[1], eod_cas_se[1], aod_cas_se[1], erd_cas_se[1]]

gcausal_means_stage_2 = [gspd_cas_mean[1], geod_cas_mean[1], gaod_cas_mean[1], gerd_cas_mean[1]]
gcausal_se_stage_2 = [gspd_cas_se[1], geod_cas_se[1], gaod_cas_se[1], gerd_cas_se[1]]


local_mean_stage_3 = [spd_mean[2], eod_mean[2], aod_mean[2], erd_mean[2]]
local_se_stage_3 = [spd_se[2], eod_se[2], aod_se[2], erd_se[2]]

global_mean_stage_3 = [gspd_mean[2], geod_mean[2], gaod_mean[2], gerd_mean[2]]
global_se_stage_3 = [gspd_se[2], geod_se[2], gaod_se[2], gerd_se[2]]

# Causal Local and Global for SelectKBest
causal_means_stage_3 = [spd_cas_mean[2], eod_cas_mean[2], aod_cas_mean[2], erd_cas_mean[2]]
causal_se_stage_3 = [spd_cas_se[2], eod_cas_se[2], aod_cas_se[2], erd_cas_se[2]]

gcausal_means_stage_3 = [gspd_cas_mean[2], geod_cas_mean[2], gaod_cas_mean[2], gerd_cas_mean[2]]
gcausal_se_stage_3 = [gspd_cas_se[2], geod_cas_se[2], gaod_cas_se[2], gerd_cas_se[2]]
# Create subplots with 2 rows and 2 columns
fig, axs = plt.subplots(3, 2, figsize=(12, 10))

# Plot 1: Statistical Local and Global Fairness for MV
plot_fairness_metrics(axs[0, 0], local_mean_stage_1, global_mean_stage_1, local_se_stage_1, global_se_stage_1, 
                      'Statistical Local and Global Fairness - SS', metric_labels)

# Plot 2: Causal Local and Global Fairness for MV
plot_fairness_metrics(axs[0, 1], causal_means_stage_1, gcausal_means_stage_1, causal_se_stage_1, gcausal_se_stage_1, 
                      'Causal Local and Global Fairness - SS', metric_labels)

# Plot 3: Statistical Local and Global Fairness for FE
plot_fairness_metrics(axs[1, 0], local_mean_stage_2, global_mean_stage_2, local_se_stage_2, global_se_stage_2, 
                      'Statistical Local and Global Fairness - Custom FE ', metric_labels)

# Plot 4: Causal Local and Global Fairness for FE
plot_fairness_metrics(axs[1, 1], causal_means_stage_2, gcausal_means_stage_2, causal_se_stage_2, gcausal_se_stage_2, 
                      'Causal Local and Global Fairness - Custom FE', metric_labels)

# Plot 3: Statistical Local and Global Fairness for FE
plot_fairness_metrics(axs[2, 0], local_mean_stage_3, global_mean_stage_3, local_se_stage_3, global_se_stage_3, 
                      'Statistical Local and Global Fairness - Stratify', metric_labels)

# Plot 4: Causal Local and Global Fairness for FE
plot_fairness_metrics(axs[2, 1], causal_means_stage_3, gcausal_means_stage_3, causal_se_stage_3, gcausal_se_stage_3, 
                      'Causal Local and Global Fairness - Stratify', metric_labels)

# Adjust layout and show plot
plt.tight_layout()
plt.show()


In [ ]:

def compute_propensity_scores(X, treatment_column):
    imputer = SimpleImputer(strategy='most_frequent')
    
    # Convert to dense if X is sparse
    if isinstance(X, csr_matrix):
        X = X.toarray()
    
    X_imputed = imputer.fit_transform(X)
    
    # Use the provided treatment column directly
    treatment = np.where((treatment_column == 4) | (treatment_column == 'White'), 1, 0)

    # Use the imputed covariate matrix without the treatment column
    X_covariates = X_imputed

    # One-hot encode the covariate matrix (if needed)
    X_encoded = pd.get_dummies(pd.DataFrame(X_covariates), drop_first=True).values

    # Estimate propensity scores using logistic regression
    lr = LogisticRegression()
    lr.fit(X_encoded, treatment)
    propensity_scores = lr.predict_proba(X_encoded)[:, 1]

    return propensity_scores, treatment

def perform_matching(propensity_scores, treatment):
    # Define treated and control indices based on the treatment variable
    treated_indices = np.where((treatment == 1))[0]  # 'White' or 4 are considered treated
    control_indices = np.where((treatment == 0))[0]  # All others are control

    # Fit the nearest neighbors model on control propensity scores
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(propensity_scores[control_indices].reshape(-1, 1))

    # Find the nearest neighbors for treated units
    distances, indices = nn.kneighbors(propensity_scores[treated_indices].reshape(-1, 1))

    # Map control indices to matched treated indices
    matched_control_indices = control_indices[indices.flatten()]

    # Combine treated and matched control indices
    matched_indices = np.concatenate([treated_indices, matched_control_indices])

    # Ensure indices are unique and within bounds
    matched_indices = np.unique(matched_indices)
    matched_indices = matched_indices[matched_indices < len(treatment)]

    return matched_indices

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np



def flip_race(attribute):
    return ['Non-White' if x == '4' else 'White' for x in attribute]

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, handle_unknown='ignore'):
        self.columns = columns
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        self.encoders = {}
        if isinstance(X, pd.DataFrame):
            for col in self.columns:
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[col])
        else:
            for i, col in enumerate(self.columns):
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[:, i])
        return self

    def transform(self, X):
        X_copy = X.copy()
        if isinstance(X_copy, pd.DataFrame):
            for col in self.columns:
                X_copy[col] = X_copy[col].map(lambda s: self.encoders[col].transform([s])[0] 
                                              if s in self.encoders[col].classes_ else -1)
        else:
            for i, col in enumerate(self.columns):
                X_copy[:, i] = np.array([self.encoders[col].transform([s])[0] 
                                         if s in self.encoders[col].classes_ else -1 
                                         for s in X_copy[:, i]])
        return X_copy

# The rest of your code remains the same

# Paths to train and test datasets
train_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.data'
test_path = 'C:/Users/Saadia/FairPreprocessing/data/adult/adult.test'

# Define column names and missing values
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 
                'capital-loss', 'hours-per-week', 'native-country', 'income-per-year']
na_values = ['?']

# Read the train and test datasets
train = pd.read_csv(train_path, header=None, names=column_names, 
                    skipinitialspace=True, na_values=na_values)
test = pd.read_csv(test_path, header=None, names=column_names,
                   skipinitialspace=True, na_values=na_values)

# Map target variable
train['income-per-year'] = train['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})
test['income-per-year'] = test['income-per-year'].map({'<=50K': 0, '>50K': 1, '<=50K.': 0, '>50K.': 1})

# Combine train and test data for re-splitting
combined_data = pd.concat([train, test])

# Drop rows with missing target values
combined_data = combined_data.dropna(subset=['income-per-year'])

X_combined = combined_data.drop('income-per-year', axis=1)
y_combined = combined_data['income-per-year']

# Categorical and numerical columns
categorical_cols = X_combined.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_combined.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Convert all categorical columns to strings to avoid type issues
X_combined[categorical_cols] = X_combined[categorical_cols].astype(str)

# Initialize lists to store results
spd_list = []
eod_list = []
aod_list = []
erd_list = []

spd_cas_list = []
eod_cas_list = []
aod_cas_list = []
erd_cas_list = []

gspd_list = []
geod_list = []
gaod_list = []
gerd_list = []

gspd_cas_list = []
geod_cas_list = []
gaod_cas_list = []
gerd_cas_list = []

# Perform 10 iterations
for i in range(10):
    print(f"Iteration {i+1}")
    # Randomly split the data into training and test sets
    X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42, stratify=y_combined)
    X_train_no_strat, X_test_no_strat, y_train_no_strat, y_test_no_strat = train_test_split(X_combined, y_combined, test_size=0.3, random_state=42)

    pipeline_1.fit(X_train_strat, y_train_strat)
    pipeline_2.fit(X_train_strat, y_train_strat)
    pipeline_3.fit(X_train_strat, y_train_strat)

    # For P4
    pipeline_4.fit(X_train_no_strat, y_train_no_strat)

    y_pred_1 = pipeline_1.predict(X_test_strat)
    y_pred_2 = pipeline_2.predict(X_test_strat)
    y_pred_3 = pipeline_3.predict(X_test_strat)

# For P4
    y_pred_4 = pipeline_4.predict(X_test_no_strat)
    
    
    res1 = guard_and_mitigate_if_needed(
        pipeline=pipeline_1,
        X_train=X_train, y_train=y_train,
        X_test=X_test,   y_test=y_test,
        sensitive_attr_test=s_test,
        thresholds=thresholds,
        mitigations_cfg=mitigations_cfg,  # the same list you pass to run_mitigations_and_predict
        context=common_context,           # e.g., {"s_train": s_train, ...}
        fairness_compute_type="global",   # or "local"
        y2_scores=None                    # provide scores if your fairness function needs them
    )
    y_pred_1n = res1["y_pred_final"]

    res2 = guard_and_mitigate_if_needed(
        pipeline=pipeline_2, X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
        thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
    )
    y_pred_2n = res2["y_pred_final"]

    res3 = guard_and_mitigate_if_needed(
        pipeline=pipeline_3, X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
        thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
    )
    y_pred_3n = res3["y_pred_final"]

    res4 = guard_and_mitigate_if_needed(
        pipeline=pipeline_4, X_train=X_train, y_train=y_train,
        X_test=X_test, y_test=y_test, sensitive_attr_test=s_test,
        thresholds=thresholds, mitigations_cfg=mitigations_cfg, context=common_context
    )
    y_pred_4n = res["y_pred_final"]
    
    SPD_1, EOD_1, AOD_1, ERD_1 = Fairness_Metrics_Computation(y_pred_1n, y_pred_2n, y_test, X_test['sex'],compute_type='local')
    SPD_2, EOD_2, AOD_2, ERD_2 = Fairness_Metrics_Computation(y_pred_1n, y_pred_3n, y_test, X_test['sex'],compute_type='local')
    SPD_3, EOD_3, AOD_3, ERD_3 = Fairness_Metrics_Computation(y_pred_1n, y_pred_4n, y_test, X_test['sex'],compute_type='local')
    
    # Store results
    spd_list.append((SPD_1, SPD_2, SPD_3))
    eod_list.append((EOD_1, EOD_2, EOD_3))
    aod_list.append((AOD_1, AOD_2, AOD_3))
    erd_list.append((ERD_1, ERD_2, ERD_3))
    
    gSPD_1, gEOD_1, gAOD_1, gERD_1 = Fairness_Metrics_Computation(y_pred_1n y_pred_2n, y_test, X_test['sex'],compute_type='global')
    gSPD_2, gEOD_2, gAOD_2, gERD_2 = Fairness_Metrics_Computation(y_pred_1n, y_pred_3n, y_test, X_test['sex'],compute_type='global')
    gSPD_3, gEOD_3, gAOD_3, gERD_3 = Fairness_Metrics_Computation(y_pred_1n, y_pred_4n, y_test, X_test['sex'],compute_type='global')
    
    # Store results
    gspd_list.append((gSPD_1, gSPD_2, gSPD_3))
    geod_list.append((gEOD_1, gEOD_2, gEOD_3))
    gaod_list.append((gAOD_1, gAOD_2, gAOD_3))
    gerd_list.append((gERD_1, gERD_2, gERD_3))
       
    propensity_scores1, treatment1 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices1 = perform_matching(propensity_scores1, treatment1)

    propensity_scores2, treatment2 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices2 = perform_matching(propensity_scores2, treatment2)

    propensity_scores3, treatment3 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices3 = perform_matching(propensity_scores3, treatment3)

    propensity_scores4, treatment4 = compute_propensity_scores(X_test, X_test['race'])
    matched_indices4 = perform_matching(propensity_scores4, treatment4)
    
    common_matched_indices = set(matched_indices1).intersection(matched_indices2).intersection(matched_indices3).intersection(matched_indices4)
    common_matched_indices = list(common_matched_indices)
    common_matched_indices.sort()

    y1_matched = y_pred_1n[common_matched_indices]
    y2_matched = y_pred_2n[common_matched_indices]
    y3_matched = y_pred_3n[common_matched_indices]
    y4_matched = y_pred_4[common_matched_indices]
    
    y_test_matched = y_test.reset_index(drop=True)[common_matched_indices]
    sex_matched = X_test['race'].reset_index(drop=True)[common_matched_indices]
    
    # Compute causal fairness metrics
    SPD_1_cas, EOD_1_cas, AOD_1_cas, ERD_1_cas = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched ,compute_type='local')
    SPD_2_cas, EOD_2_cas, AOD_2_cas, ERD_2_cas = Fairness_Metrics_Computation(y1_matched, y3_matched, y_test_matched, sex_matched,compute_type='local')
    SPD_3_cas, EOD_3_cas, AOD_3_cas, ERD_3_cas = Fairness_Metrics_Computation(y1_matched, y4_matched, y_test_matched, sex_matched,compute_type='local')
    
    # Store causal results
    spd_cas_list.append((SPD_1_cas, SPD_2_cas, SPD_3_cas))
    eod_cas_list.append((EOD_1_cas, EOD_2_cas, EOD_3_cas))
    aod_cas_list.append((AOD_1_cas, AOD_2_cas, AOD_3_cas))
    erd_cas_list.append((ERD_1_cas, ERD_2_cas, ERD_3_cas))

    gSPD_1_cas, gEOD_1_cas, gAOD_1_cas, gERD_1_cas = Fairness_Metrics_Computation(y1_matched, y2_matched, y_test_matched, sex_matched ,compute_type='global')
    gSPD_2_cas, gEOD_2_cas, gAOD_2_cas, gERD_2_cas = Fairness_Metrics_Computation(y1_matched, y3_matched, y_test_matched, sex_matched,compute_type='global')
    gSPD_3_cas, gEOD_3_cas, gAOD_3_cas, gERD_3_cas = Fairness_Metrics_Computation(y1_matched, y4_matched, y_test_matched, sex_matched,compute_type='global')
    
    # Store causal results
    gspd_cas_list.append((gSPD_1_cas, gSPD_2_cas, gSPD_3_cas))
    geod_cas_list.append((gEOD_1_cas, gEOD_2_cas, gEOD_3_cas))
    gaod_cas_list.append((gAOD_1_cas, gAOD_2_cas, gAOD_3_cas))
    gerd_cas_list.append((gERD_1_cas, gERD_2_cas, gERD_3_cas))
    


In [ ]:
def compute_means_and_errors(array):
    mean = array.mean(axis=0) * 0.1
    se = (array.std(axis=0) / np.sqrt(array.shape[0])) * 0.1
    return np.round(mean, 3), np.round(se, 3)


spd_array = np.array(spd_list)
eod_array = np.array(eod_list)
aod_array = np.array(aod_list)
erd_array = np.array(erd_list)

gspd_array = np.array(gspd_list)
geod_array = np.array(geod_list)
gaod_array = np.array(gaod_list)
gerd_array = np.array(gerd_list)

spd_cas_array = np.array(spd_cas_list)
eod_cas_array = np.array(eod_cas_list)
aod_cas_array = np.array(aod_cas_list)
erd_cas_array = np.array(erd_cas_list)

gspd_cas_array = np.array(gspd_cas_list)
geod_cas_array = np.array(geod_cas_list)
gaod_cas_array = np.array(gaod_cas_list)
gerd_cas_array = np.array(gerd_cas_list)


spd_mean, spd_se = compute_means_and_errors(spd_array)
eod_mean, eod_se = compute_means_and_errors(eod_array)
aod_mean, aod_se = compute_means_and_errors(aod_array)
erd_mean, erd_se = compute_means_and_errors(erd_array)

gspd_mean, gspd_se = compute_means_and_errors(gspd_array)
geod_mean, geod_se = compute_means_and_errors(geod_array)
gaod_mean, gaod_se = compute_means_and_errors(gaod_array)
gerd_mean, gerd_se = compute_means_and_errors(gerd_array)

spd_cas_mean, spd_cas_se = compute_means_and_errors(spd_cas_array)
eod_cas_mean, eod_cas_se = compute_means_and_errors(eod_cas_array)
aod_cas_mean, aod_cas_se = compute_means_and_errors(aod_cas_array)
erd_cas_mean, erd_cas_se = compute_means_and_errors(erd_cas_array)

gspd_cas_mean, gspd_cas_se = compute_means_and_errors(gspd_cas_array)
geod_cas_mean, geod_cas_se = compute_means_and_errors(geod_cas_array)
gaod_cas_mean, gaod_cas_se = compute_means_and_errors(gaod_cas_array)
gerd_cas_mean, gerd_cas_se = compute_means_and_errors(gerd_cas_array)


import numpy as np
import matplotlib.pyplot as plt

# Assuming your fairness metrics arrays are correctly formatted
# Metric labels
metric_labels = ['SPD', 'EOD', 'AOD', 'ERD']

# Colors for the bars
colors = ['blue', 'orange']
labels = ['Local', 'Global']

# Plot function for fairness metrics
def plot_fairness_metrics(ax, local_means, global_means, local_se, global_se, title, metric_labels):
    x = np.arange(len(metric_labels))  # label locations
    width = 0.35  # the width of the bars

    rects1 = ax.bar(x - width/2, local_means, width, label='Local', yerr=local_se, capsize=5)
    rects2 = ax.bar(x + width/2, global_means, width, label='Global', yerr=global_se, capsize=5)

    # Add labels, title, and custom ticks
    ax.set_xlabel('Fairness Metrics')
    ax.set_ylabel('Values')
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(metric_labels)
    ax.legend()

# Replace these with the actual means and errors for statistical and causal fairness
# Statistical Local and Global for PCA
local_mean_stage_1 = [spd_mean[0], eod_mean[0], aod_mean[0], erd_mean[0]]
local_se_stage_1 = [spd_se[0], eod_se[0], aod_se[0], erd_se[0]]

global_mean_stage_1 = [gspd_mean[0], geod_mean[0], gaod_mean[0], gerd_mean[0]]
global_se_stage_1 = [gspd_se[0], geod_se[0], gaod_se[0], gerd_se[0]]

# Causal Local and Global for PCA
causal_means_stage_1 = [spd_cas_mean[0], -eod_cas_mean[0], aod_cas_mean[0], erd_cas_mean[0]]
causal_se_stage_1 = [spd_cas_se[0], eod_cas_se[0], aod_cas_se[0], erd_cas_se[0]]

gcausal_means_stage_1 = [gspd_cas_mean[0], geod_cas_mean[0], gaod_cas_mean[0], gerd_cas_mean[0]]
gcausal_se_stage_1 = [gspd_cas_se[0], geod_cas_se[0], gaod_cas_se[0], gerd_cas_se[0]]

# Statistical Local and Global for SelectKBest
local_mean_stage_2 = [spd_mean[1], eod_mean[1], aod_mean[1], erd_mean[1]]
local_se_stage_2 = [spd_se[1], eod_se[1], aod_se[1], erd_se[1]]

global_mean_stage_2 = [gspd_mean[1], geod_mean[1], gaod_mean[1], gerd_mean[1]]
global_se_stage_2 = [gspd_se[1], geod_se[1], gaod_se[1], gerd_se[1]]

# Causal Local and Global for SelectKBest
causal_means_stage_2 = [spd_cas_mean[1], eod_cas_mean[1], aod_cas_mean[1], erd_cas_mean[1]]
causal_se_stage_2 = [spd_cas_se[1], eod_cas_se[1], aod_cas_se[1], erd_cas_se[1]]

gcausal_means_stage_2 = [gspd_cas_mean[1], geod_cas_mean[1], gaod_cas_mean[1], gerd_cas_mean[1]]
gcausal_se_stage_2 = [gspd_cas_se[1], geod_cas_se[1], gaod_cas_se[1], gerd_cas_se[1]]


local_mean_stage_3 = [spd_mean[2], eod_mean[2], aod_mean[2], erd_mean[2]]
local_se_stage_3 = [spd_se[2], eod_se[2], aod_se[2], erd_se[2]]

global_mean_stage_3 = [gspd_mean[2], geod_mean[2], gaod_mean[2], gerd_mean[2]]
global_se_stage_3 = [gspd_se[2], geod_se[2], gaod_se[2], gerd_se[2]]

# Causal Local and Global for SelectKBest
causal_means_stage_3 = [spd_cas_mean[2], eod_cas_mean[2], aod_cas_mean[2], erd_cas_mean[2]]
causal_se_stage_3 = [spd_cas_se[2], eod_cas_se[2], aod_cas_se[2], erd_cas_se[2]]

gcausal_means_stage_3 = [gspd_cas_mean[2], geod_cas_mean[2], gaod_cas_mean[2], gerd_cas_mean[2]]
gcausal_se_stage_3 = [gspd_cas_se[2], geod_cas_se[2], gaod_cas_se[2], gerd_cas_se[2]]
# Create subplots with 2 rows and 2 columns
fig, axs = plt.subplots(3, 2, figsize=(12, 10))

# Plot 1: Statistical Local and Global Fairness for MV
plot_fairness_metrics(axs[0, 0], local_mean_stage_1, global_mean_stage_1, local_se_stage_1, global_se_stage_1, 
                      'Statistical Local and Global Fairness - SS', metric_labels)

# Plot 2: Causal Local and Global Fairness for MV
plot_fairness_metrics(axs[0, 1], gcausal_means_stage_1, causal_means_stage_1, gcausal_se_stage_1, causal_se_stage_1, 
                      'Causal Local and Global Fairness - SS', metric_labels)

# Plot 3: Statistical Local and Global Fairness for FE
plot_fairness_metrics(axs[1, 0], local_mean_stage_2, global_mean_stage_2, local_se_stage_2, global_se_stage_2, 
                      'Statistical Local and Global Fairness - Custom FE ', metric_labels)

# Plot 4: Causal Local and Global Fairness for FE
plot_fairness_metrics(axs[1, 1], gcausal_means_stage_2, causal_means_stage_2, causal_se_stage_2, gcausal_se_stage_2, 
                      'Causal Local and Global Fairness - Custom FE', metric_labels)

# Plot 3: Statistical Local and Global Fairness for FE
plot_fairness_metrics(axs[2, 0], local_mean_stage_3, global_mean_stage_3, local_se_stage_3, global_se_stage_3, 
                      'Statistical Local and Global Fairness - Stratify', metric_labels)

# Plot 4: Causal Local and Global Fairness for FE
plot_fairness_metrics(axs[2, 1], causal_means_stage_3, gcausal_means_stage_3, causal_se_stage_3, gcausal_se_stage_3, 
                      'Causal Local and Global Fairness - Stratify', metric_labels)

# Adjust layout and show plot
plt.tight_layout()
plt.show()
